In [2]:
import cython

In [3]:
%load_ext cython

In [19]:
%%cython -f -a
# distutils: language = c++
# distutils: extra_compile_args = /std:c++20
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True, initializedcheck=False


import numpy as np
cimport numpy as np
from libcpp cimport bool as cpp_bool
from scipy.linalg.cython_lapack cimport zgesv
from scipy.special.cython_special cimport spherical_jn
from libc.stdio cimport printf, sprintf
from libc.stdlib cimport exit, EXIT_FAILURE
from libc.math cimport NAN, pi, isfinite, isinf, isnan, copysign, \
    sqrt, fabs, signbit, exp, cos, sin, log, log1p, ldexp, atan2, frexp, ceil, M_PI
from libc.string cimport strcpy
from libc.float cimport DBL_MAX, DBL_MIN, DBL_MANT_DIG
cdef double PI_DBL = M_PI
cdef double complex cmplx_NAN = NAN + 1.0j * NAN
cdef double complex cmplx_zero = 0.0 + 0.0j
from libcpp.limits cimport numeric_limits
cdef double INF_DBL = numeric_limits[double].infinity()
cdef double EPS_DBL = numeric_limits[double].epsilon()
cdef double NAN_DBL = numeric_limits[double].quiet_NaN()
from CyRK cimport cysolve_ivp, CySolverResult, CySolveOutput, DiffeqFuncType, PreEvalFunc
from CyRK.utils.utils cimport allocate_mem, reallocate_mem, free_mem
from CyRK.utils.vector cimport vector

ctypedef CySolverResult* CySolveResultPtr
ctypedef vector[CySolveResultPtr] EOSSolutionVec

from scipy.constants import G as G_
cdef double G = G_

cdef extern from * nogil:
    """
    #include <cstring>
    #include <complex>
    #include <vector>

    const int MAX_NUM_Y = 6;
    const int MAX_NUM_Y_REAL = 12; // Maximum number of y values counting both the real and imaginary portions.
    
    
    class RadialSolutionStorageCC
    {
        // Public Attributes
    public:
        size_t num_slices;
        size_t total_size;
        char message[256] = { };
        char* message_ptr = &message[0];
        bool success      = false;
        char num_ytypes   = 0;
    
        // Radial solution attributes
        std::vector<double> full_solution_vec = std::vector<double>(0);
        double* full_solution_ptr = NULL;
    
        // Equation of state attributes
        char eos_message[256] = { };
        char* eos_message_ptr = &message[0];
        bool eos_success      = false;
        std::vector<double> eos_properties_vec = std::vector<double>(0);
        double* eos_properties_ptr = NULL;
        double* gravity_ptr   = NULL;
        double* pressure_ptr  = NULL;
        double* density_ptr   = NULL;
        double* shear_mod_ptr = NULL;
        double* bulk_mod_ptr  = NULL;
    
        // Love number attributes
        std::vector<double> complex_love_vec = std::vector<double>(0);
        double* complex_love_ptr = NULL;
    
        // Constructors and methods
        RadialSolutionStorageCC() { };
        virtual ~RadialSolutionStorageCC() { };
        RadialSolutionStorageCC(size_t num_slices, char num_ytypes);
        
        void set_message(const char* new_message);
        void find_love(double surface_gravity);
    };

    void find_love_cf(
        double* complex_love_numbers_ptr,  // These are double pointers that pointer to a double complex array.
        double* surface_solutions_ptr,     // Same as above
        double surface_gravity);


    void find_love_cf(
        double* complex_love_numbers_ptr,  // These are double pointers that pointer to a double complex array.
        double* surface_solutions_ptr,     // Same as above
        double surface_gravity)
    {
        const double y1_real = surface_solutions_ptr[0];
        const double y1_imag = surface_solutions_ptr[1];
        const double y3_real = surface_solutions_ptr[4];
        const double y3_imag = surface_solutions_ptr[5];
        const double y5_real = surface_solutions_ptr[8];
        const double y5_imag = surface_solutions_ptr[9];
    
        // Calculate Love and Shida numbers
        // Note Im(k2) = -Im(y5) (Henning & Hurford 2014 eq. A9), opposite convention of Tobie et al. (2005, eqs. 9 & 36)
        // And k2 = |-y5-1| (Roberts & Nimmo 2008 equation A8), not 1-y5 as in Henning & Hurford (2014) equation A9.
         
        // Okay, some clarification on this. It looks like VS04 that HH14 is based on used a different convention for y5,
        //      Tobie05's y5 = -y5 of SV; we follow that format here.
    
        // Love k
        complex_love_numbers_ptr[0] = y5_real - 1.0;
        complex_love_numbers_ptr[1] = y5_imag;
    
        // Love h
        complex_love_numbers_ptr[2] = y1_real * surface_gravity;
        complex_love_numbers_ptr[3] = y1_imag * surface_gravity;
    
        // Shida l
        complex_love_numbers_ptr[4] = y3_real * surface_gravity;
        complex_love_numbers_ptr[5] = y3_imag * surface_gravity;
    }


    
    RadialSolutionStorageCC::RadialSolutionStorageCC(
            size_t num_slices,
            char num_ytypes) :
                num_slices(num_slices),
                num_ytypes(num_ytypes)
    {
        // Initialize status
        this->success = false;
        this->set_message("RadialSolutionStorageCC has not been updated with data yet.");
    
        // Find total data size
        this->total_size = (size_t)MAX_NUM_Y_REAL * this->num_slices * (size_t)this->num_ytypes;
    
        // Reserve space in the storage vectors.
        // These are double vectors but store double complex values so we need to double the amount of storage.
        // This is done already by using MAX_NUM_Y_REAL over MAX_NUM_Y
        this->full_solution_vec.reserve(this->total_size);
        this->full_solution_ptr = &this->full_solution_vec[0];
    
        // Three Love numbers are stored for each requested y-type.
        // These are also double vectors storing double complex values, double the storage.
        this->complex_love_vec.reserve(2 * 3 * this->num_ytypes);
        this->complex_love_ptr = &this->complex_love_vec[0];
    
        // The radial solution class will also store the output of the equation of state solver. 
        // Setup the eos solution vector and associated pointers.
        // The vector will be 7 x total slices
        // Array 0: Gravity
        // Array 1: Pressure
        // Array 2: Density
        // Double Array 3a + 3b: Shear Mod (real, imag)
        // Double Array 4a + 4b: Bulk Mod (real, imag)
        this->eos_properties_vec.reserve(7 * this->num_slices);
        this->eos_properties_ptr = &this->full_solution_vec[0];
        double* gravity_ptr   = &this->eos_properties_ptr[0];
        double* pressure_ptr  = &this->eos_properties_ptr[this->num_slices];
        double* density_ptr   = &this->eos_properties_ptr[2 * this->num_slices];
        double* shear_mod_ptr = &this->eos_properties_ptr[3 * this->num_slices];
        double* bulk_mod_ptr  = &this->eos_properties_ptr[5 * this->num_slices]; // The jump from 3 to 5 here is because shear_mod_ptr (and this bulk_mod_ptr) is twice as long as the others.
    }
    
    void RadialSolutionStorageCC::find_love(double surface_gravity)
    {   
        if (this->success) [[likely]]
        {
            const size_t top_slice_i   = this->num_slices - 1;
            const size_t num_output_ys = MAX_NUM_Y_REAL * this->num_ytypes;
    
            // For each y-type, use the solution at the surface of the planet to find each Love number.
            double surface_solutions[MAX_NUM_Y_REAL] = { };
            double* surface_solutions_ptr            = &surface_solutions[0];
    
            for (char ytype_i = 0; ytype_i < this->num_ytypes; ytype_i++)
            {
                // Pull out surface solutions for this y-type
                for (int y_i = 0; y_i < MAX_NUM_Y_REAL; y_i++)
                {
                    const size_t lhs_y_index = ytype_i * MAX_NUM_Y_REAL + y_i;
                    surface_solutions_ptr[y_i] = this->full_solution_ptr[top_slice_i * num_output_ys + lhs_y_index];
                }
                find_love_cf(&this->complex_love_ptr[2 * 3 * ytype_i], surface_solutions_ptr, surface_gravity);
            }
        }
        else
        {
            // Could pass a new message to update the state but it will overwrite any error message that is already there.
            // TODO: Think about doing an append or logging system in the future.
            // this->set_message("Can not update Love number values when solution is not complete or successful.")
        }
    }
    
    void RadialSolutionStorageCC::set_message(const char* new_message_ptr)
    {
        std::strcpy(this->message_ptr, new_message_ptr);
    }

    
    """
    void find_love_cf(
        double* complex_love_numbers_ptr,
        double* surface_solutions_ptr,
        double surface_gravity)

    cdef cppclass RadialSolutionStorageCC:
        RadialSolutionStorageCC()
        RadialSolutionStorageCC(
            size_t num_slices,
            char num_ytypes
            )
        size_t num_slices
        size_t total_size
        char[256] message
        char* message_ptr
        cpp_bool success
        char num_ytypes

        vector[double] complex_love_vec
        double* complex_love_ptr
        vector[double] full_solution_vec
        double* full_solution_ptr
        char[256] eos_message
        char* eos_message_ptr
        cpp_bool eos_success

        vector[double] eos_properties_vec
        double* eos_properties_ptr
        double* gravity_ptr
        double* pressure_ptr
        double* density_ptr
        double* shear_mod_ptr
        double* bulk_mod_ptr

        void find_love(double surface_gravity) noexcept nogil
        void set_message(const char* new_message) noexcept nogil


# Maximum size for array building
cdef size_t MAX_NUM_Y      = 6
cdef size_t MAX_NUM_Y_REAL = 2 * MAX_NUM_Y
cdef size_t MAX_NUM_SOL    = 3

cdef int DBL_MANT_DIG_INT = <int>DBL_MANT_DIG

cdef double SQRT2 = 1.414213562373095048801688724209698079  # sqrt 2
cdef double LOGE2 = 0.693147180559945309417232121458176568  # log_e 2

# We risk spurious overflow for components >= DBL_MAX / (1 + sqrt(2)).
cdef double SQRT2_INV = 1. / (1.0 + SQRT2)
cdef double THRESH    = SQRT2_INV * DBL_MAX
cdef double DBL_MAX_4 = 0.25 * DBL_MAX

# scaled_cexp precison constant
#if @precision@ == 1
# precision for float
cdef int SCALED_CEXP_K_F  = 235
# precision for double
cdef int SCALED_CEXP_K_D  = 1799
# precision for long double
cdef int SCALED_CEXP_K_LD = 19547

cdef double SCALED_K_LOGE2_D = SCALED_CEXP_K_D * LOGE2

cdef float SCALED_CEXP_LOWERF = 88.722839
cdef float SCALED_CEXP_UPPERF = 192.69492
cdef double SCALED_CEXP_LOWER  = 710.47586007394386
cdef double SCALED_CEXP_UPPER  = 1454.9159319953251
cdef long double SCALED_CEXP_LOWERL = 11357.216553474703895
cdef long double SCALED_CEXP_UPPERL = 22756.021937783004509


cdef inline double complex cf_build_dblcmplx(const double a, const double b) noexcept nogil:
    cdef double complex result
    cdef double* result_dbl_ptr = <double *> &result
    result_dbl_ptr[0] = a
    result_dbl_ptr[1] = b

    return result

cdef double cf_cabs(double complex z) noexcept nogil:
    cdef double z_real = z.real
    cdef double z_imag = z.imag
    return sqrt((z_real * z_real) + (z_imag * z_imag))

cdef double cf_hypot(const double x, const double y) noexcept nogil:

    cdef double yx
    cdef double temp
    cdef double x_abs, y_abs

    if isinf(x) or isinf(y):
        return INF_DBL
    
    if isnan(x) or isnan(y):
        return NAN_DBL

    x_abs = fabs(x)
    y_abs = fabs(y)
    if x_abs < y_abs:
        temp  = x_abs
        x_abs = y_abs
        y_abs = temp

    if x_abs == 0.:
        return 0.
    else:
        yx = y_abs / x_abs
        return x_abs * sqrt(1. + yx * yx)


cdef double complex cf_csqrt(const double complex z) noexcept nogil:
    cdef double complex result
    cdef double t
    cdef char scale

    cdef double z_real = z.real
    cdef double z_imag = z.imag

    # Handle special cases.
    if z_imag == 0.0:
        if z_real == 0.0:
            return cf_build_dblcmplx(0., 0.)
        elif z_real > 0.0:
            return cf_build_dblcmplx(sqrt(z_real), 0.0)

    if isinf(z_imag):
        # Return inf +- inf (where sign is the same as input)
        return cf_build_dblcmplx(INF_DBL, z_imag)

    if isnan(z_real):
        # Return NAN_DBL + NAN_DBL i
        return cf_build_dblcmplx(NAN_DBL, NAN_DBL)

    if isinf(z_real):
        # csqrt(-inf + NAN_DBL i) = NAN_DBL +- inf i
        # csqrt(-inf + y i)   = 0   +  inf i
        # csqrt(inf + NAN_DBL i)  = inf +  NAN_DBL i
        # csqrt(inf + y i)    = inf +  0 i
        
        if signbit(z_real):
            # Negative z_real
            if isnan(z_imag):
                return cf_build_dblcmplx(NAN_DBL, INF_DBL)
            else:
                return cf_build_dblcmplx(0., INF_DBL)
        else:
            if isnan(z_imag):
                return cf_build_dblcmplx(INF_DBL, NAN_DBL)
            else:
                return cf_build_dblcmplx(INF_DBL, 0.)

    # The remaining special case (b is NAN_DBL) is handled just fine by the normal code path below.
    # Scale to avoid overflow.
    if (fabs(z_real) >= THRESH) or (fabs(z_imag) >= THRESH):
        z_real *= 0.25
        z_imag *= 0.25
        scale = 1
    else:
        scale = 0

    # Algorithm 312, CACM vol 10, Oct 1967.
    if z_real >= 0.0:
        t = sqrt((z_real + cf_hypot(z_real, z_imag)) * 0.5)
        result = cf_build_dblcmplx(t, (z_imag / (2.0 * t)))
    else:
        t = sqrt((-z_real + cf_hypot(z_real, z_imag)) * 0.5)
        result = cf_build_dblcmplx((fabs(z_imag) / (2.0 * t)), copysign(t, z_imag))

    # Rescale.
    if scale == 1:
        return cf_build_dblcmplx(result.real * 2.0, result.imag)
    else:
        return result



cdef double complex cf_z_calc(
        double complex x_squared,
        unsigned char degree_l
        ) noexcept nogil:
    """ Calculates the z function using spherical Bessel function, see Eq. B14 of KMN15.

    References
    ----------
    TS72 Eqs. 96, 97
    KMN15 Eq. B14

    Parameters
    ----------
    x_squared : double complex
        Expression passed to the Bessel function.
    degree_l : unsigned int
        Tidal harmonic order.

    Returns
    -------
    z : double complex
        Result
    """

    # QUESTION: (Issue #42) The recrusision formula shown in TS72 and KMN15 (See TS72 Eq. 97) does not reproduce the full version
    # of this function (TS72 Eq. 96). I am finding it to be off by about 30% at l = 2 and then the error improves to 
    # 10% at l=10. Perhaps thre is a problem with the implementation.
    # In any case, we will not use the recrusive formula here.

    # Have found that the Taylor expansion works well when x_square is small and is faster.
    cdef double l2            = <double>degree_l * 2.0
    
    # TODO: There is an issue with scipy 1.14.X+ on MacOS where the conversion between Py_ssize_t and long is not being accepted. See issue #65
    cdef long degree_l_ssizet = <long>degree_l

    cdef double complex x, z
    cdef double l2_3, l2_5, l2_7, l2_9, l2_11

    if cf_cabs(x_squared) > 0.1:
        # Use real function
        x = cf_csqrt(x_squared)
        z =  x * spherical_jn(degree_l_ssizet + 1, x) / spherical_jn(degree_l_ssizet, x)
    else:
        # Use Taylor series; JPR derived this on 2024-02-05
        l2_3  = l2 + 3.0
        l2_5  = l2 + 5.0
        l2_7  = l2 + 7.0
        l2_9  = l2 + 9.0
        l2_11 = l2 + 11.0
        #   Numerator                              | Denominator
        z = \
            (x_squared                             / l2_3) + \
            (x_squared**2                          / (l2_3**2 * l2_5)) + \
            (x_squared**4 * 2.                     / (l2_3**3 * l2_5 * l2_7)) + \
            (x_squared**6 * (27. + 10. * degree_l) / (l2_3**4 * l2_5**2 * l2_7 * l2_9)) + \
            (x_squared**8 * (90. + 28. * degree_l) / (l2_3**5 * l2_5**2 * l2_7 * l2_9 * l2_11))
    
    return z


cdef void cf_takeuchi_phi_psi(
        double complex z2,
        unsigned char degree_l,
        double complex* phi_ptr,
        double complex* phi_lplus1_ptr,
        double complex* psi_ptr,
        ) noexcept nogil:
    """ Calculate the two (plus one) functions used to find initial conditions for shooting method.

    References
    ----------
    TS72 Eq. 103

    """

    # Floating point errors prevent us from using the exact definition of these functions (See Issue #41)
    # Instead we use the limiting version of the functions.
    # However, we leave the full definition at the bottom of this function for reference.

    cdef double complex z4   = z2 * z2
    cdef double complex z6   = z4 * z2
    cdef double complex z8   = z6 * z2
    cdef double complex z10  = z8 * z2
    cdef double l    = <double>degree_l
    cdef double l2   = 2. * l
    cdef double l_3  = 3.  + l2
    cdef double l_5  = 5.  + l2
    cdef double l_7  = 7.  + l2
    cdef double l_9  = 9.  + l2
    cdef double l_11 = 11. + l2
    cdef double l_13 = 13. + l2

    # Series expansion was done in `Takeuchi Starting Conditions.nb` on 2024-01-31 by JPR.
    phi_ptr[0] = (
         1. + 
        -z2  / (2.    * l_3) + 
         z4  / (8.    * l_3 * l_5) + 
        -z6  / (48.   * l_3 * l_5 * l_7) + 
         z8  / (384.  * l_3 * l_5 * l_7 * l_9) + 
        -z10 / (3840. * l_3 * l_5 * l_7 * l_9 * l_11)
    )
    
    # Note that the l+1 function has skips a denominator term than the function above.
    phi_lplus1_ptr[0] = (
         1. + 
        -z2  / (2.    * l_5) + 
         z4  / (8.    * l_5 * l_7) + 
        -z6  / (48.   * l_5 * l_7 * l_9) + 
         z8  / (384.  * l_5 * l_7 * l_9 * l_11) + 
        -z10 / (3840. * l_5 * l_7 * l_9 * l_11 * l_13)
    )
    
    psi_ptr[0] = (
         1. + 
        -z2  / (4.     * l_5) + 
        # RECORD: Error? TS72 quotes the next term as z4 / (12. * (5. + 2. * l) * (7 + 2. * l)); factor of 2 off in denom.
         z4  / (24.    * l_5 * l_7) + 
        -z6  / (192.   * l_5 * l_7 * l_9) + 
         z8  / (1920.  * l_5 * l_7 * l_9 * l_11) +
        -z10 / (23040. * l_5 * l_7 * l_9 * l_11 * l_13)
     )
    
    # Full version
    # cdef char lp1 = degree_l + 1
    # cdef double l_dbl_factorial   = cf_double_factorial(2 * degree_l + 1)
    # cdef double lp1_dbl_factorial = cf_double_factorial(2 * lp1 + 1)
    
    # cdef double complex z    = cf_csqrt(z2)
    # cdef double complex zl   = cf_cipow(z, degree_l)
    # cdef double complex zlp1 = cf_cipow(z, lp1)

    # phi_ptr[0]        = l_dbl_factorial * (spherical_jn(degree_l, z) / zl)
    # phi_lplus1_ptr[0] = lp1_dbl_factorial * (spherical_jn(lp1, z) / zlp1)
    # psi_ptr[0]        = (2. * (2. * degree_l + 3.) / (z * z)) * (1. - phi_ptr[0])

    # DEBUG: Uncomment the below and comment the above to use the method that TidalPy v0.4.0 used for this function.
    # Left here for comparison/debug purpouses.
    # cdef double degree_lp1 = degree_l + 1.
    # cdef double complex z4 = z2 * z2

    # phi_ptr[0] = 1. - \
    #       z2 / (2. * (2. * degree_l + 3.)) + \
    #       z4 / (8. * (2. * degree_l + 3.) * (2. * degree_l + 5.))
    # phi_lplus1_ptr[0] = 1. - \
    #              z2 / (2. * (2. * degree_lp1 + 3.)) + \
    #              z4 / (8. * (2. * degree_lp1 + 3.) * (2. * degree_lp1 + 5.))
    # psi_ptr[0] = 1. - \
    #       z2 / (4. * (2. * degree_l + 5.)) + \
    #       z4 / (12. * (2. * degree_l + 5.) * (2. * degree_l + 7.))

########################################################################################################################
#### Solid Layers
########################################################################################################################


cdef void cf_takeuchi_solid_dynamic_compressible(
        double frequency,
        double radius,
        double density,
        double complex bulk_modulus,
        double complex shear_modulus,
        unsigned char degree_l,
        double G_to_use,
        ssize_t num_ys, 
        double complex* starting_conditions_ptr
        ) noexcept nogil:
    """ Calculate the starting guess at the bottom of a solid layer using the dynamic assumption.

    This function uses the Takeuchi and Saito 1972 equations (Eq. 95-101).

    Using the dynamic assumption in a solid layer results in three independent solutions for the radial derivatives.

    These independent solutions allow for a general tidal harmonic l, for dynamic tides (w != 0), compressibility, and
       bulk and shear dissipation.

    References
    ----------
    TS72

    Parameters
    ----------
    frequency : double
        Forcing frequency (for spin-synchronous tides this is the orbital motion) [rad s-1]
    radius : double
        Radius where the radial functions are calculated. [m]
    density : double
        Density at `radius` [kg m-3]
    bulk_modulus : double complex
        Bulk modulus (can be complex for dissipation) at `radius` [Pa]
    shear_modulus : double complex
        Shear modulus (can be complex for dissipation) at `radius` [Pa]
    degree_l : unsigned char
        Tidal harmonic order.
    G_to_use : double
        Gravitational constant. Provide a non-dimensional version if the rest of the inputs are non-dimensional.
    num_ys : ssize_t
        Number of radial solutions for this layer type.
    starting_conditions_ptr : double complex*, <Output>
        Desired starting conditions for this layer.
        Three independent solid guesses (sn1, sn2, sn3)

    """

    # Convert compressibility parameters
    cdef double complex lame = bulk_modulus - (2. / 3.) * shear_modulus

    # Constants
    cdef double gamma           = 4. * pi * G_to_use * density / 3.
    cdef double dynamic_term    = frequency * frequency
    cdef double complex alpha2  = (lame + 2. * shear_modulus) / density
    cdef double complex beta2   = shear_modulus / density

    # Optimizations
    cdef double r_inverse    = 1. / radius
    cdef double r2           = radius * radius
    cdef double degree_l_dbl = <double> degree_l
    cdef double lp1          = degree_l_dbl + 1.
    cdef double lm1          = degree_l_dbl - 1.
    cdef double dlp1         = 2. * degree_l_dbl + 1.
    cdef double dlp3         = 2. * degree_l_dbl + 3.
    cdef double llp1         = degree_l_dbl * lp1

    # Helper functions
    # See Eq. 99 of TS72
    cdef double complex k2_quad_pos  = (dynamic_term / beta2) + ((dynamic_term + 4. * gamma) / alpha2)
    cdef double complex k2_quad_neg  = (dynamic_term / beta2) - ((dynamic_term + 4. * gamma) / alpha2)
    cdef double complex k2_quad      = (k2_quad_neg * k2_quad_neg) + ((4. * llp1 * gamma**2) / (alpha2 * beta2))
    cdef double complex k2_quad_sqrt = cf_csqrt(k2_quad)

    # QUESTION: (Issue #43) TS74 has these flipped compared to KMN15. Going with TS74 for this func.
    # See the -/+ order in TS72 EQ. 99
    cdef size_t neg_index = 0
    cdef size_t pos_index = 1
    cdef double complex k2_neg = (1. / 2.) * (k2_quad_pos - k2_quad_sqrt)
    cdef double complex k2_pos = (1. / 2.) * (k2_quad_pos + k2_quad_sqrt)

    cdef double complex f_pos = (beta2 * k2_pos - dynamic_term) / gamma
    cdef double complex f_neg = (beta2 * k2_neg - dynamic_term) / gamma

    cdef double complex h_pos = f_pos - lp1
    cdef double complex h_neg = f_neg - lp1

    # Calculate Takeuchi and Saito functions
    cdef double complex z2_pos = k2_pos * r2
    cdef double complex z2_neg = k2_neg * r2

    cdef double complex phi_pos, phi_lp1_pos, psi_pos
    cdef double complex phi_neg, phi_lp1_neg, psi_neg
    cf_takeuchi_phi_psi(z2_pos, degree_l, &phi_pos, &phi_lp1_pos, &psi_pos)
    cf_takeuchi_phi_psi(z2_neg, degree_l, &phi_neg, &phi_lp1_neg, &psi_neg)

    # See Eq. 102 in TS72
    
    # y1, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 0] = \
        ((-radius**lp1) / dlp3) * (.5 * degree_l_dbl * h_pos * psi_pos + f_pos * phi_lp1_pos)
    starting_conditions_ptr[neg_index * num_ys + 0] = \
        ((-radius**lp1) / dlp3) * (.5 * degree_l_dbl * h_neg * psi_neg + f_neg * phi_lp1_neg)
    starting_conditions_ptr[2 * num_ys + 0] = degree_l_dbl * radius**lm1
    
    # y2, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 1] = \
        -(lame + 2. * shear_modulus) * radius**degree_l_dbl * f_pos * phi_pos + \
        (shear_modulus * radius**degree_l_dbl / dlp3) * (
            -degree_l_dbl * lm1 * h_pos * psi_pos + 2. * (2. * f_pos + llp1) * phi_lp1_pos
            )
    starting_conditions_ptr[neg_index * num_ys + 1] = \
        -(lame + 2. * shear_modulus) * radius**degree_l_dbl * f_neg * phi_neg + \
        (shear_modulus * radius**degree_l_dbl / dlp3) * (
            -degree_l_dbl * lm1 * h_neg * psi_neg + 2. * (2. * f_neg + llp1) * phi_lp1_neg
            )
    starting_conditions_ptr[2 * num_ys + 1] = \
        2. * shear_modulus * degree_l_dbl * lm1 * radius**(degree_l_dbl - 2.)
    
    # y3, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 2] = \
        (-radius**lp1 / dlp3) * (0.5 * h_pos * psi_pos - phi_lp1_pos)
    starting_conditions_ptr[neg_index * num_ys + 2] = \
        (-radius**lp1 / dlp3) * (0.5 * h_neg * psi_neg - phi_lp1_neg)
    starting_conditions_ptr[2 * num_ys + 2] = \
        radius**lm1
    
    # y4, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 3] = \
        shear_modulus * radius**degree_l_dbl * (
            phi_pos - (1. / dlp3) * (lm1 * h_pos * psi_pos + 2. * (f_pos + 1.) * phi_lp1_pos)
            )
    starting_conditions_ptr[neg_index * num_ys + 3] = \
        shear_modulus * radius**degree_l_dbl * (
            phi_neg - (1. / dlp3) * (lm1 * h_neg * psi_neg + 2. * (f_neg + 1.) * phi_lp1_neg)
            )
    starting_conditions_ptr[2 * num_ys + 3] = \
        2. * shear_modulus * lm1 * radius**(degree_l_dbl - 2.)
    
    # y5, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 4] = \
        radius**(degree_l_dbl + 2.) * (
            (alpha2 * f_pos - lp1 * beta2) / r2 - (3. * gamma * f_pos / (2. * dlp3)) * psi_pos
            )
    starting_conditions_ptr[neg_index * num_ys + 4] = \
        radius**(degree_l_dbl + 2.) * (
            (alpha2 * f_neg - lp1 * beta2) / r2 - (3. * gamma * f_neg / (2. * dlp3)) * psi_neg
            )
    starting_conditions_ptr[2 * num_ys + 4] = \
        (degree_l_dbl * gamma - dynamic_term) * radius**degree_l_dbl
    
    # y6, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 5] = \
        dlp1 * r_inverse * starting_conditions_ptr[0 * num_ys + 4] + \
        (3. * degree_l_dbl * gamma * h_pos * radius**lp1 / (2. * dlp3)) * psi_pos
    starting_conditions_ptr[neg_index * num_ys + 5] = \
        dlp1 * r_inverse * starting_conditions_ptr[1 * num_ys + 4] + \
        (3. * degree_l_dbl * gamma * h_neg * radius**lp1 / (2. * dlp3)) * psi_neg
    starting_conditions_ptr[2 * num_ys + 5] = \
        dlp1 * r_inverse * starting_conditions_ptr[2 * num_ys + 4] - \
        3. * degree_l_dbl * gamma * radius**lm1


cdef void cf_takeuchi_solid_static_compressible(
        double radius,
        double density,
        double complex bulk_modulus,
        double complex shear_modulus,
        unsigned char degree_l,
        double G_to_use,
        ssize_t num_ys, 
        double complex* starting_conditions_ptr
        ) noexcept nogil:
    """ Calculate the starting guess at the bottom of a solid layer using the static assumption.

    This function uses the Takeuchi and Saito 1972 equations (Eq. 95-101).

    Using the static assumption in a solid layer results in two independent solutions for the radial derivative.

    These independent solutions allow for a general tidal harmonic l, for static tides (w = 0), compressibility, and
       bulk and shear dissipation.

    References
    ----------
    TS72

    Parameters
    ----------
    radius : double
        Radius where the radial functions are calculated. [m]
    density : double
        Density at `radius` [kg m-3]
    bulk_modulus : double complex
        Bulk modulus (can be complex for dissipation) at `radius` [Pa]
    shear_modulus : double complex
        Shear modulus (can be complex for dissipation) at `radius` [Pa]
    degree_l : unsigned char
        Tidal harmonic order.
    G_to_use : double
        Gravitational constant. Provide a non-dimensional version if the rest of the inputs are non-dimensional.
    num_ys : ssize_t
        Number of radial solutions for this layer type.
    starting_conditions_ptr : double complex*, <Output>
        Desired starting conditions for this layer.
        Three independent solid guesses (sn1, sn2, sn3)
    """

    # Convert compressibility parameters
    cdef double complex lame = bulk_modulus - (2. / 3.) * shear_modulus

    # Constants
    cdef double gamma          = 4. * pi * G_to_use * density / 3.
    cdef double complex alpha2 = (lame + 2. * shear_modulus) / density
    cdef double complex beta2  = shear_modulus / density

    # Optimizations
    cdef double r_inverse    = 1. / radius
    cdef double r2           = radius * radius
    cdef double degree_l_dbl = <double> degree_l
    cdef double lp1          = degree_l_dbl + 1.
    cdef double lm1          = degree_l_dbl - 1.
    cdef double dlp1         = 2. * degree_l_dbl + 1.
    cdef double dlp3         = 2. * degree_l_dbl + 3.
    cdef double llp1         = degree_l_dbl * lp1

    # Helper functions
    # See Eq. 99 of TS72
    cdef double complex k2_quad_pos  = (4. * gamma / alpha2)
    cdef double complex k2_quad      = k2_quad_pos**2 + ((4. * llp1 * gamma**2) / (alpha2 * beta2))
    cdef double complex k2_quad_sqrt = cf_csqrt(k2_quad)

    # QUESTION: (Issue #43) TS74 has these flipped compared to KMN15. Going with TS74 for this func.
    # See the -/+ order in TS72 EQ. 99
    cdef size_t neg_index = 0
    cdef size_t pos_index = 1
    cdef double complex k2_neg = (1. / 2.) * (k2_quad_pos - k2_quad_sqrt)
    cdef double complex k2_pos = (1. / 2.) * (k2_quad_pos + k2_quad_sqrt)

    cdef double complex f_pos = (beta2 * k2_pos) / gamma
    cdef double complex f_neg = (beta2 * k2_neg) / gamma

    cdef double complex h_pos = f_pos - lp1
    cdef double complex h_neg = f_neg - lp1

    # Calculate Takeuchi and Saito functions
    cdef double complex z2_pos = k2_pos * r2
    cdef double complex z2_neg = k2_neg * r2

    cdef double complex phi_pos, phi_lp1_pos, psi_pos
    cdef double complex phi_neg, phi_lp1_neg, psi_neg
    cf_takeuchi_phi_psi(z2_pos, degree_l, &phi_pos, &phi_lp1_pos, &psi_pos)
    cf_takeuchi_phi_psi(z2_neg, degree_l, &phi_neg, &phi_lp1_neg, &psi_neg)

    # See Eq. 102 in TS72
    # y1, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 0] = \
        ((-radius**lp1) / dlp3) * (0.5 * degree_l * h_pos * psi_pos + f_pos * phi_lp1_pos)
    starting_conditions_ptr[neg_index * num_ys + 0] = \
        ((-radius**lp1) / dlp3) * (0.5 * degree_l * h_neg * psi_neg + f_neg * phi_lp1_neg)
    starting_conditions_ptr[2 * num_ys + 0] = \
        degree_l * radius**lm1

    # y2, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 1] = \
        -(lame + 2. * shear_modulus) * radius**degree_l * f_pos * phi_pos + \
        (shear_modulus * radius**degree_l / dlp3) * (
            -degree_l * lm1 * h_pos * psi_pos + 2. * (2. * f_pos + llp1) * phi_lp1_pos
            )
    starting_conditions_ptr[neg_index * num_ys + 1] = \
        -(lame + 2. * shear_modulus) * radius**degree_l * f_neg * phi_neg + \
        (shear_modulus * radius**degree_l / dlp3) * (
            -degree_l * lm1 * h_neg * psi_neg + 2. * (2. * f_neg + llp1) * phi_lp1_neg
            )
    starting_conditions_ptr[2 * num_ys + 1] = \
        2. * shear_modulus * degree_l * lm1 * radius**(degree_l - 2.)

    # y3, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 2] = \
        (-radius**lp1 / dlp3) * (0.5 * h_pos * psi_pos - phi_lp1_pos)
    starting_conditions_ptr[neg_index * num_ys + 2] = \
        (-radius**lp1 / dlp3) * (0.5 * h_neg * psi_neg - phi_lp1_neg)
    starting_conditions_ptr[2 * num_ys + 2] = \
        radius**lm1

    # y4, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 3] = \
        shear_modulus * radius**degree_l * \
        (phi_pos - (1. / dlp3) * (lm1 * h_pos * psi_pos + 2. * (f_pos + 1.) * phi_lp1_pos))
    starting_conditions_ptr[neg_index * num_ys + 3] = \
        shear_modulus * radius**degree_l * \
        (phi_neg - (1. / dlp3) * (lm1 * h_neg * psi_neg + 2. * (f_neg + 1.) * phi_lp1_neg))
    starting_conditions_ptr[2 * num_ys + 3] = \
        2. * shear_modulus * lm1 * radius**(degree_l - 2.)

    # y5, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 4] = \
        radius**(degree_l + 2.) * \
        ((alpha2 * f_pos - lp1 * beta2) / r2 - (3. * gamma * f_pos / (2. * dlp3)) * psi_pos)
    starting_conditions_ptr[neg_index * num_ys + 4] = \
        radius**(degree_l + 2.) * \
        ((alpha2 * f_neg - lp1 * beta2) / r2 - (3. * gamma * f_neg / (2. * dlp3)) * psi_neg)
    starting_conditions_ptr[2 * num_ys + 4] = \
        degree_l * gamma * radius**degree_l

    # y6, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 5] = \
        dlp1 * r_inverse * starting_conditions_ptr[0 * num_ys + 4] + \
        (3. * degree_l * gamma * h_pos * radius**lp1 / (2. * dlp3)) * psi_pos
    starting_conditions_ptr[neg_index * num_ys + 5] = \
        dlp1 * r_inverse * starting_conditions_ptr[1 * num_ys + 4] + \
        (3. * degree_l * gamma * h_neg * radius**lp1 / (2. * dlp3)) * psi_neg
    starting_conditions_ptr[2 * num_ys + 5] = \
        dlp1 * r_inverse * starting_conditions_ptr[2 * num_ys + 4] - \
        3. * degree_l * gamma * radius**lm1


########################################################################################################################
#### Liquid Layers
########################################################################################################################


cdef void cf_takeuchi_liquid_dynamic_compressible(
        double frequency,
        double radius,
        double density,
        double complex bulk_modulus,
        unsigned char degree_l,
        double G_to_use,
        ssize_t num_ys, 
        double complex* starting_conditions_ptr
        ) noexcept nogil:
    """ Calculate the starting guess at the bottom of a liquid layer using the dynamic assumption.

    This function uses the Takeuchi and Saito 1972 equations (Eq. 95-101).

    Using the dynamic assumption in a liquid layer results in two independent solutions for the radial derivatives.

    These independent solutions allow for a general tidal harmonic l, for dynamic tides (w != 0), compressibility, and
       bulk dissipation (no shear dissipation within liquid layers).

    References
    ----------
    TS72

    Parameters
    ----------
    frequency : double
        Forcing frequency (for spin-synchronous tides this is the orbital motion) [rad s-1]
    radius : double
        Radius where the radial functions are calculated. [m]
    density : double
        Density at `radius` [kg m-3]
    bulk_modulus : double complex
        Bulk modulus (can be complex for dissipation) at `radius` [Pa]
    degree_l : unsigned char
        Tidal harmonic order.
    G_to_use : double
        Gravitational constant. Provide a non-dimensional version if the rest of the inputs are non-dimensional.
    num_ys : ssize_t
        Number of radial solutions for this layer type.
    starting_conditions_ptr : double complex*, <Output>
        Desired starting conditions for this layer.
        Two independent liquid guesses (sn1, sn2)
    """

    # Convert compressibility parameters
    cdef double complex lame = bulk_modulus

    # Constants
    cdef double dynamic_term   = frequency * frequency
    cdef double gamma          = 4. * pi * G_to_use * density / 3.
    cdef double complex alpha2 = lame / density

    # Optimizations
    cdef double r_inverse    = 1. / radius
    cdef double r2           = radius * radius
    cdef double degree_l_dbl = <double> degree_l
    cdef double lp1          = degree_l_dbl + 1.
    cdef double lm1          = degree_l_dbl - 1.
    cdef double dlp1         = 2. * degree_l_dbl + 1.
    cdef double dlp3         = 2. * degree_l_dbl + 3.
    cdef double llp1         = degree_l_dbl * lp1

    # Helper functions
    # k2 h and f no longer depend on k2. See Eq. 101 of TS72
    cdef double f  = -dynamic_term / gamma
    cdef double h  = f - lp1
    cdef double complex k2 = (1. / alpha2) * (dynamic_term + 4. * gamma - llp1 * gamma**2 / dynamic_term)

    # Calculate Takeuchi and Saito functions
    cdef double complex z = k2 * r2
    cdef double complex phi, phi_lp1, psi
    cf_takeuchi_phi_psi(z, degree_l, &phi, &phi_lp1, &psi)

    # Found by setting mu=0 in Eq. 102 of TS72
    # y1, solutions 1--2
    starting_conditions_ptr[0 * num_ys + 0] = \
        ((-radius**lp1) / dlp3) * (0.5 * degree_l * h * psi + f * phi_lp1)
    starting_conditions_ptr[1 * num_ys + 0] = \
        degree_l * radius**lm1

    # y2, solutions 1--2
    starting_conditions_ptr[0 * num_ys + 1] = \
        -lame * radius**degree_l * f * phi
    starting_conditions_ptr[1 * num_ys + 1] = \
        0.0

    # y5, solutions 1--2
    starting_conditions_ptr[0 * num_ys + 2] = \
        radius**(degree_l + 2.) * ((alpha2 * f / r2) - (3. * gamma * f / (2. * dlp3)) * psi)
    starting_conditions_ptr[1 * num_ys + 2] = \
        (degree_l * gamma - dynamic_term) * radius**degree_l

    # y6, solutions 1--2
    starting_conditions_ptr[0 * num_ys + 3] = \
        dlp1 * r_inverse * starting_conditions_ptr[0 * num_ys + 2] + (3. * degree_l * gamma * h * radius**lp1 / (2. * dlp3)) * psi
    starting_conditions_ptr[1 * num_ys + 3] = \
        dlp1 * r_inverse * starting_conditions_ptr[1 * num_ys + 2] - 3. * degree_l * gamma * radius**lm1

cdef void cf_saito_liquid_static_inccompressible(
        double radius,
        unsigned char degree_l,
        ssize_t num_ys, 
        double complex* starting_conditions_ptr
        ) noexcept nogil:
    """ Calculate the initial guess at the bottom of a liquid layer using the static assumption.

    This function uses the Saito 1974 equations (Eq. 19).

    Using the static assumption in a liquid layer results in one independent solutions for the radial derivative.

    These independent solution allow for a general tidal harmonic l, for static tides (w = 0).
    However, compressibility and all dissipation dependence is lost due to no dependence on bulk or shear moduli.


    References
    ----------
    S74

    Parameters
    ----------
    radius : double
        Radius where the radial functions are calculated. [m]
    degree_l : unsigned char
        Tidal harmonic order.
    num_ys : ssize_t
        Number of radial solutions for this layer type.
    starting_conditions_ptr : double complex*, <Output>
        Desired starting conditions for this layer.
        One independent liquid guess (sn1)
    """

    # See Eq. 19 in Saito 1974
    # # y5 solution 0
    starting_conditions_ptr[0] = radius**degree_l

    # # y7 solution 0
    starting_conditions_ptr[1] = 2. * (degree_l - 1.) * radius**(degree_l - 1.)


cdef void cf_kamata_solid_dynamic_compressible(
        double frequency,
        double radius,
        double density,
        double complex bulk_modulus,
        double complex shear_modulus,
        unsigned int degree_l,
        double G_to_use,
        ssize_t num_ys, 
        double complex* starting_conditions_ptr
        ) noexcept nogil:
    """ Calculate the starting guess at the bottom of a solid layer using the dynamic assumption.

    This function uses the Kamata et al (2015; JGR:P) equations (Eq. B1-B16).

    Using the dynamic assumption in a solid layer results in three independent solutions for the radial derivatives.

    These independent solutions allow for a general tidal harmonic l, for dynamic tides (w != 0), compressibility, and
       bulk and shear dissipation.

    References
    ----------
    KMN15 Eqs. B1-B16

    Parameters
    ----------
    frequency : double
        Forcing frequency (for spin-synchronous tides this is the orbital motion) [rad s-1]
    radius : double
        Radius where the radial functions are calculated. [m]
    density : double
        Density at `radius` [kg m-3]
    bulk_modulus : double complex
        Bulk modulus (can be complex for dissipation) at `radius` [Pa]
    shear_modulus : double complex
        Shear modulus (can be complex for dissipation) at `radius` [Pa]
    degree_l : unsigned char
        Tidal harmonic order.
    G_to_use : double
        Gravitational constant. Provide a non-dimensional version if the rest of the inputs are non-dimensional.
    num_ys : ssize_t
        Number of radial solutions for this layer type.
    starting_conditions_ptr : double complex*, <Output>
        Desired starting conditions for this layer.
        Three independent solid guesses (sn1, sn2, sn3)
    """

    # Convert compressibility parameters
    cdef double complex lame = bulk_modulus - (2. / 3.) * shear_modulus

    # Constants
    cdef double gamma          = 4. * pi * G_to_use * density / 3.
    cdef double dynamic_term   = frequency * frequency
    cdef double complex alpha2 = (lame + 2. * shear_modulus) / density
    cdef double complex beta2  = shear_modulus / density

    # Optimizations
    cdef double r_inverse    = 1. / radius
    cdef double r2_inverse   = r_inverse * r_inverse
    cdef double r2           = radius * radius
    cdef double degree_l_dbl = <double> degree_l
    cdef double lp1          = degree_l_dbl + 1.0
    cdef double dlp1         = 2.0 * degree_l_dbl + 1.0
    cdef double llp1         = degree_l_dbl * lp1

    # Helper functions
    cdef double complex k2_quad_pos = (dynamic_term / beta2) + ((dynamic_term + 4. * gamma) / alpha2)
    cdef double complex k2_quad_neg = (dynamic_term / beta2) - ((dynamic_term + 4. * gamma) / alpha2)
    cdef double complex k2_quad     = (k2_quad_neg * k2_quad_neg) + \
        ((4. * degree_l * (degree_l + 1) * (gamma * gamma)) / (alpha2 * beta2))

    # QUESTION: (Issue #43) KMN15 has these flipped compared to TS72. Going with  KMN15 for this func.
    cdef size_t neg_index = 1
    cdef size_t pos_index = 0
    cdef double complex k2_quad_sqrt = cf_csqrt(k2_quad)
    cdef double complex k2_pos = (1. / 2.) * (k2_quad_pos + k2_quad_sqrt)
    cdef double complex k2_neg = (1. / 2.) * (k2_quad_pos - k2_quad_sqrt)

    cdef double complex f_k2_pos = (beta2 * k2_pos - dynamic_term) / gamma
    cdef double complex f_k2_neg = (beta2 * k2_neg - dynamic_term) / gamma

    cdef double complex h_k2_pos = f_k2_pos - lp1
    cdef double complex h_k2_neg = f_k2_neg - lp1

    cdef double complex z_k2_pos = cf_z_calc(k2_pos * r2, degree_l=degree_l)
    cdef double complex z_k2_neg = cf_z_calc(k2_neg * r2, degree_l=degree_l)

    # See Eqs. B1-B12 of KMN15
    
    # y1, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 0] = \
        -f_k2_pos * z_k2_pos * r_inverse
    starting_conditions_ptr[neg_index * num_ys + 0] = \
        -f_k2_neg * z_k2_neg * r_inverse
    starting_conditions_ptr[2 * num_ys + 0] = \
        degree_l_dbl * r_inverse
    
    # y2, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 1] = \
        -density * f_k2_pos * alpha2 * k2_pos + (2. * shear_modulus * r2_inverse) * (2. * f_k2_pos + llp1) * z_k2_pos
    starting_conditions_ptr[neg_index * num_ys + 1] = \
        -density * f_k2_neg * alpha2 * k2_neg + (2. * shear_modulus * r2_inverse) * (2. * f_k2_neg + llp1) * z_k2_neg
    starting_conditions_ptr[2 * num_ys + 1] = \
        2. * shear_modulus * degree_l_dbl * (degree_l_dbl - 1) * r2_inverse
    
    # y3, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 2] = \
        z_k2_pos * r_inverse
    starting_conditions_ptr[neg_index * num_ys + 2] = \
        z_k2_neg * r_inverse
    starting_conditions_ptr[2 * num_ys + 2] = \
        r_inverse
    
    # y4, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 3] = \
        shear_modulus * k2_pos - (2. * shear_modulus * r2_inverse) * (f_k2_pos + 1.) * z_k2_pos
    starting_conditions_ptr[neg_index * num_ys + 3] = \
        shear_modulus * k2_neg - (2. * shear_modulus * r2_inverse) * (f_k2_neg + 1.) * z_k2_neg
    starting_conditions_ptr[2 * num_ys + 3] = \
        2. * shear_modulus * (degree_l_dbl - 1.) * r2_inverse
    
    # y5, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 4] = \
        3. * gamma * f_k2_pos - h_k2_pos * (degree_l_dbl * gamma - dynamic_term)
    starting_conditions_ptr[neg_index * num_ys + 4] = \
        3. * gamma * f_k2_neg - h_k2_neg * (degree_l_dbl * gamma - dynamic_term)
    starting_conditions_ptr[2 * num_ys + 4] = \
        degree_l * gamma - dynamic_term
    
    # y6, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 5] = \
        dlp1 * starting_conditions_ptr[pos_index * num_ys + 4] * r_inverse
    starting_conditions_ptr[neg_index * num_ys + 5] = \
        dlp1 * starting_conditions_ptr[neg_index * num_ys + 4] * r_inverse
    starting_conditions_ptr[2 * num_ys + 5] = \
        dlp1 * starting_conditions_ptr[2 * num_ys + 4] * r_inverse - (3. * degree_l_dbl * gamma * r_inverse)





cdef void cf_kamata_solid_static_compressible(
        double radius,
        double density,
        double complex bulk_modulus,
        double complex shear_modulus,
        unsigned int degree_l,
        double G_to_use,
        ssize_t num_ys, 
        double complex* starting_conditions_ptr
        ) noexcept nogil:
    """ Calculate the starting guess at the bottom of a solid layer using the static assumption.

    This function uses the Kamata et al. (2015; JGR:P) equations (Eq. B1-B16).

    Using the static assumption in a solid layer results in three independent solutions for the radial derivatives.

    These independent solutions allow for a general tidal harmonic l, for static tides (w = 0), compressibility, and
       bulk and shear dissipation.

    References
    ----------
    KMN15

    Parameters
    ----------
    radius : double
        Radius where the radial functions are calculated. [m]
    density : double
        Density at `radius` [kg m-3]
    bulk_modulus : double complex
        Bulk modulus (can be complex for dissipation) at `radius` [Pa]
    shear_modulus : double complex
        Shear modulus (can be complex for dissipation) at `radius` [Pa]
    degree_l : unsigned char
        Tidal harmonic order.
    G_to_use : double
        Gravitational constant. Provide a non-dimensional version if the rest of the inputs are non-dimensional.
    num_ys : ssize_t
        Number of radial solutions for this layer type.
    starting_conditions_ptr : double complex*, <Output>
        Desired starting conditions for this layer.
        Three independent solid guesses (sn1, sn2, sn3)
    """

    # Convert compressibility parameters
    cdef double complex lame = bulk_modulus - (2. / 3.) * shear_modulus

    # Constants
    cdef double gamma          = 4. * pi * G_to_use * density / 3.
    cdef double complex alpha2 = (lame + 2. * shear_modulus) / density
    cdef double complex beta2  = shear_modulus / density

    # Optimizations
    cdef double r_inverse    = 1. / radius
    cdef double r2_inverse   = r_inverse * r_inverse
    cdef double r2           = radius * radius
    cdef double degree_l_dbl = <double>degree_l
    cdef double lp1          = degree_l_dbl + 1.
    cdef double dlp1         = 2.0 * degree_l_dbl + 1.0
    cdef double llp1         = degree_l_dbl * lp1

    # Helper functions
    cdef double complex k2_quad_pos = 4. * gamma / alpha2
    cdef double complex k2_quad_neg = -k2_quad_pos
    cdef double complex k2_quad = k2_quad_neg**2 + ((4. * degree_l * lp1 * gamma**2) / (alpha2 * beta2))

    # QUESTION: (Issue #43) KMN15 has these flipped compared to TS72. Going with  KMN15 for this func.
    cdef size_t neg_index = 1
    cdef size_t pos_index = 0
    cdef double complex k2_quad_sqrt = cf_csqrt(k2_quad)
    cdef double complex k2_pos = (1. / 2.) * (k2_quad_pos + k2_quad_sqrt)
    cdef double complex k2_neg = (1. / 2.) * (k2_quad_pos - k2_quad_sqrt)
    
    cdef double complex f_k2_pos = beta2 * k2_pos / gamma
    cdef double complex f_k2_neg = beta2 * k2_neg / gamma

    cdef double complex h_k2_pos = f_k2_pos - lp1
    cdef double complex h_k2_neg = f_k2_neg - lp1

    cdef double complex z_k2_pos = cf_z_calc(k2_pos * r2, degree_l=degree_l)
    cdef double complex z_k2_neg = cf_z_calc(k2_neg * r2, degree_l=degree_l)

    # See Eqs. B1-B12 of KMN15
    
    # y1, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 0] = \
        -f_k2_pos * z_k2_pos * r_inverse
    starting_conditions_ptr[neg_index * num_ys + 0] = \
        -f_k2_neg * z_k2_neg * r_inverse
    starting_conditions_ptr[2 * num_ys + 0] = \
        degree_l * r_inverse
    
    # y2, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 1] = \
        -density * f_k2_pos * alpha2 * k2_pos + (2. * shear_modulus * r2_inverse) * (2. * f_k2_pos + llp1) * z_k2_pos
    starting_conditions_ptr[neg_index * num_ys + 1] = \
        -density * f_k2_neg * alpha2 * k2_neg + (2. * shear_modulus * r2_inverse) * (2. * f_k2_neg + llp1) * z_k2_neg
    starting_conditions_ptr[2 * num_ys + 1] = \
        2. * shear_modulus * degree_l * (degree_l - 1) * r2_inverse
    
    # y3, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 2] = \
        z_k2_pos * r_inverse
    starting_conditions_ptr[neg_index * num_ys + 2] = \
        z_k2_neg * r_inverse
    starting_conditions_ptr[2 * num_ys + 2] = \
        r_inverse
    
    # y4, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 3] = \
        shear_modulus * k2_pos - (2. * shear_modulus * r2_inverse) * (f_k2_pos + 1.) * z_k2_pos
    starting_conditions_ptr[neg_index * num_ys + 3] = \
        shear_modulus * k2_neg - (2. * shear_modulus * r2_inverse) * (f_k2_neg + 1.) * z_k2_neg
    starting_conditions_ptr[2 * num_ys + 3] = \
        2. * shear_modulus * (degree_l - 1.) * r2_inverse
    
    # y5, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 4] = \
        3. * gamma * f_k2_pos - h_k2_pos * (degree_l * gamma)
    starting_conditions_ptr[neg_index * num_ys + 4] = \
        3. * gamma * f_k2_neg - h_k2_neg * (degree_l * gamma)
    starting_conditions_ptr[2 * num_ys + 4] = \
        degree_l * gamma
    
    # y6, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 5] = \
        dlp1 * starting_conditions_ptr[pos_index * num_ys + 4] * r_inverse
    starting_conditions_ptr[neg_index * num_ys + 5] = \
        dlp1 * starting_conditions_ptr[neg_index * num_ys + 4] * r_inverse
    starting_conditions_ptr[2 * num_ys + 5] = \
        dlp1 * starting_conditions_ptr[2 * num_ys + 4] * r_inverse - (3. * degree_l * gamma * r_inverse)


cdef void cf_kamata_solid_dynamic_incompressible(
        double frequency,
        double radius,
        double density,
        double complex shear_modulus,
        unsigned int degree_l,
        double G_to_use,
        ssize_t num_ys, 
        double complex* starting_conditions_ptr
        ) noexcept nogil:
    """ Calculate the starting guess at the bottom of a solid layer using the dynamic and incompressible assumption.

    This function uses the Kamata et al. (2015; JGR:P) equations (Eq. B17-B28).

    Using the dynamic assumption in a solid layer results in three independent solutions for the radial derivatives.

    These independent solutions allow for a general tidal harmonic l, for dynamic tides (w != 0), incompressible, and
       bulk and shear dissipation.

    References
    ----------
    KMN15 Eqs. B17-28

    Parameters
    ----------
    frequency : double
        Forcing frequency (for spin-synchronous tides this is the orbital motion) [rad s-1]
    radius : double
        Radius where the radial functions are calculated. [m]
    density : double
        Density at `radius` [kg m-3]
    shear_modulus : double complex
        Shear modulus (can be complex for dissipation) at `radius` [Pa]
    degree_l : unsigned char
        Tidal harmonic order.
    G_to_use : double
        Gravitational constant. Provide a non-dimensional version if the rest of the inputs are non-dimensional.
    num_ys : ssize_t
        Number of radial solutions for this layer type.
    starting_conditions_ptr : double complex*, <Output>
        Desired starting conditions for this layer.
        Three independent solid guesses (sn1, sn2, sn3)

    """

    # Constants
    cdef double gamma         = 4. * pi * G_to_use * density / 3.
    cdef double dynamic_term  = frequency * frequency
    cdef double complex beta2 = shear_modulus / density

    # Optimizations
    cdef double r_inverse    = 1. / radius
    cdef double r2_inverse   = r_inverse * r_inverse
    cdef double r2           = radius * radius
    cdef double degree_l_dbl = <double>degree_l
    cdef double lp1          = degree_l_dbl + 1.
    cdef double lm1          = degree_l_dbl - 1.
    cdef double dlp1         = 2.0 * degree_l_dbl + 1.0
    cdef double llp1         = degree_l_dbl * lp1

    # QUESTION: (Issue #43) KMN15 has these flipped compared to TS72. Going with  KMN15 for this func.
    cdef size_t neg_index = 1
    cdef size_t pos_index = 0
    cdef double complex k2_pos   = dynamic_term / beta2
    cdef double complex f_k2_neg = -dynamic_term / gamma
    cdef double complex h_k2_neg = f_k2_neg - lp1
    cdef double complex z_k2_pos = cf_z_calc(k2_pos * r2, degree_l=degree_l)

    # See Eqs. B17-B28 of KMN15
    
    # y1, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 0] = \
        0.
    starting_conditions_ptr[neg_index * num_ys + 0] = \
        0.
    starting_conditions_ptr[2 * num_ys + 0] = \
        degree_l_dbl * r_inverse
    
    # y2, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 1] = \
        llp1 * (-density * gamma + 2. * shear_modulus * z_k2_pos * r2_inverse)
    starting_conditions_ptr[neg_index * num_ys + 1] = \
        density * ((dynamic_term / gamma) * (dynamic_term + 4. * gamma) - llp1 * gamma)
    starting_conditions_ptr[2 * num_ys + 1] = \
        2. * shear_modulus * degree_l_dbl * lm1 * r2_inverse
    
    # y3, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 2] = \
        z_k2_pos * r_inverse
    starting_conditions_ptr[neg_index * num_ys + 2] = \
        0.
    starting_conditions_ptr[2 * num_ys + 2] = \
        r_inverse
    
    # y4, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 3] = \
        shear_modulus * (dynamic_term / beta2 - 2. * r2_inverse * z_k2_pos)
    starting_conditions_ptr[neg_index * num_ys + 3] = \
        0.
    starting_conditions_ptr[2 * num_ys + 3] = \
        2. * shear_modulus * lm1 * r2_inverse
    
    # y5, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 4] = \
        lp1 * (degree_l_dbl * gamma - dynamic_term)
    starting_conditions_ptr[neg_index * num_ys + 4] = \
        (h_k2_neg - 3.) * dynamic_term - h_k2_neg * degree_l_dbl * gamma
    starting_conditions_ptr[2 * num_ys + 4] = \
        degree_l_dbl * gamma - dynamic_term
    
    # y6, solutions 1--3
    starting_conditions_ptr[pos_index * num_ys + 5] = \
        dlp1 * starting_conditions_ptr[pos_index * num_ys + 4] * r_inverse
    starting_conditions_ptr[neg_index * num_ys + 5] = \
        dlp1 * starting_conditions_ptr[neg_index * num_ys + 4] * r_inverse
    starting_conditions_ptr[2 * num_ys + 5] = \
        dlp1 * starting_conditions_ptr[2 * num_ys + 4] * r_inverse - (3. * degree_l_dbl * gamma * r_inverse)


########################################################################################################################
#### Liquid Layers
########################################################################################################################


cdef void cf_kamata_liquid_dynamic_compressible(
        double frequency,
        double radius,
        double density,
        double complex bulk_modulus,
        unsigned int degree_l,
        double G_to_use,
        ssize_t num_ys, 
        double complex* starting_conditions_ptr
        ) noexcept nogil:
    """  Calculate the starting guess at the bottom of a liquid layer using the dynamic assumption.

    This function uses the Kamata et al (2015; JGR:P) equations (Eq. B29-B37).

    Using the dynamic assumption in a liquid layer results in two independent solutions for the radial derivatives.

    These independent solutions allow for a general tidal harmonic l, for dynamic tides (w != 0), compressibility, and
       bulk dissipation (no shear dissipation within liquid layers).

    References
    ----------
    KMN15 Eq. B29-B37

    Parameters
    ----------
    frequency : double
        Forcing frequency (for spin-synchronous tides this is the orbital motion) [rad s-1]
    radius : double
        Radius where the radial functions are calculated. [m]
    density : double
        Density at `radius` [kg m-3]
    bulk_modulus : double complex
        Bulk modulus (can be complex for dissipation) at `radius` [Pa]
    degree_l : unsigned char
        Tidal harmonic order.
    G_to_use : double
        Gravitational constant. Provide a non-dimensional version if the rest of the inputs are non-dimensional.
    num_ys : ssize_t
        Number of radial solutions for this layer type.
    starting_conditions_ptr : double complex*, <Output>
        Desired starting conditions for this layer.
        Two independent liquid guesses (sn1, sn2, sn3)

    """

    # Convert compressibility parameters
    # For the liquid layer the shear modulus is zero so the 1st Lame parameter = bulk modulus
    cdef double complex lame = bulk_modulus

    # Optimizations
    cdef double dynamic_term = frequency * frequency
    cdef double r_inverse    = 1. / radius
    cdef double r2           = radius * radius
    cdef double degree_l_dbl = <double>degree_l

    # Helper functions
    cdef double gamma          = (4. * pi * G_to_use * density / 3.)
    cdef double f              = -dynamic_term / gamma
    cdef double h              = f - (degree_l_dbl + 1.)
    cdef double complex alpha2 = lame / density
    cdef double complex k2     = (1. / alpha2) * (dynamic_term + 4. * gamma - 
                                                  (degree_l_dbl * (degree_l_dbl + 1) * gamma**2 / dynamic_term))

    # See Eqs. B33--B36 in KMN15
    # y1, solutions 1--2
    starting_conditions_ptr[0 * num_ys + 0] = \
        -f * r_inverse * cf_z_calc(k2 * r2, degree_l=degree_l)
    starting_conditions_ptr[1 * num_ys + 0] = \
        degree_l_dbl * r_inverse
    
    # y2, solutions 1--2
    starting_conditions_ptr[0 * num_ys + 1] = \
        -density * (f * (dynamic_term + 4 * gamma) + degree_l_dbl * (degree_l_dbl + 1) * gamma)
    starting_conditions_ptr[1 * num_ys + 1] = \
        0.
    
    # y5, solutions 1--2
    starting_conditions_ptr[0 * num_ys + 2] = \
        3. * gamma * f - h * (degree_l_dbl * gamma - dynamic_term)
    starting_conditions_ptr[1 * num_ys + 2] = \
        degree_l_dbl * gamma - dynamic_term
    
    # y6, solutions 1--2
    starting_conditions_ptr[0 * num_ys + 3] = \
        (2. * degree_l_dbl + 1.) * starting_conditions_ptr[0 * num_ys + 2] * r_inverse
    starting_conditions_ptr[1 * num_ys + 3] = \
        ((2. * degree_l_dbl + 1.) * starting_conditions_ptr[1 * num_ys + 2] * r_inverse) - \
        ((3. * degree_l_dbl * gamma) * r_inverse)


cdef void cf_kamata_liquid_dynamic_incompressible(
        double frequency,
        double radius,
        double density,
        unsigned int degree_l,
        double G_to_use,
        ssize_t num_ys, 
        double complex* starting_conditions_ptr
        ) noexcept nogil:
    """  Calculate the starting guess at the bottom of a liquid layer using the dynamic and incompressible assumption.

    This function uses the Kamata et al. (2015; JGR:P) equations (Eq. B29-B37).

    Using the dynamic assumption in a liquid layer results in two independent solutions for the radial derivatives.

    These independent solutions allow for a general tidal harmonic l, for dynamic tides (w != 0), incompressible, and
       bulk dissipation (no shear dissipation within liquid layers).

    References
    ----------
    KMN15 Eq. B29-B37

    Parameters
    ----------
    frequency : double
        Forcing frequency (for spin-synchronous tides this is the orbital motion) [rad s-1]
    radius : double
        Radius where the radial functions are calculated. [m]
    density : double
        Density at `radius` [kg m-3]
    degree_l : unsigned char
        Tidal harmonic order.
    G_to_use : double
        Gravitational constant. Provide a non-dimensional version if the rest of the inputs are non-dimensional.
    num_ys : ssize_t
        Number of radial solutions for this layer type.
    starting_conditions_ptr : double complex*, <Output>
        Desired starting conditions for this layer.
        Two independent liquid guesses (sn1, sn2, sn3)
    """

    # Optimizations
    cdef double dynamic_term = frequency * frequency
    cdef double r_inverse    = 1. / radius
    cdef double degree_l_dbl = <double>degree_l
    cdef double lp1          = degree_l_dbl + 1.
    cdef double llp1         = degree_l_dbl * lp1
    cdef double dlp1         = 2. * degree_l_dbl + 1.

    # Helper functions
    cdef double gamma = (4. * pi * G_to_use * density / 3.)
    cdef double f     = -dynamic_term / gamma
    cdef double h     = f - lp1

    # See Eqs. B33--B36 in KMN15
    
    # y1, solutions 1--2
    starting_conditions_ptr[0 * num_ys + 0] = \
        0.
    starting_conditions_ptr[1 * num_ys + 0] = \
        degree_l_dbl * r_inverse
    
    # y2, solutions 1--2
    starting_conditions_ptr[0 * num_ys + 1] = \
        -density * (f * (dynamic_term + 4 * gamma) + llp1 * gamma)
    starting_conditions_ptr[1 * num_ys + 1] = \
        0.
    
    # y5, solutions 1--2
    starting_conditions_ptr[0 * num_ys + 2] = \
        3. * gamma * f - h * (degree_l_dbl * gamma - dynamic_term)
    starting_conditions_ptr[1 * num_ys + 2] = \
        degree_l_dbl * gamma - dynamic_term
    
    # y6, solutions 1--2
    starting_conditions_ptr[0 * num_ys + 3] = \
        dlp1 * starting_conditions_ptr[0 * num_ys + 2] * r_inverse
    starting_conditions_ptr[1 * num_ys + 3] = \
        (dlp1 * starting_conditions_ptr[1 * num_ys + 2] * r_inverse) - ((3. * degree_l_dbl * gamma) * r_inverse)



cdef void cf_find_starting_conditions(
        cpp_bool* success_ptr,
        char* message_ptr,
        int layer_type,
        bint is_static,
        bint is_incompressible,
        bint use_kamata,
        double frequency,
        double radius,
        double density,
        double complex bulk_modulus,
        double complex shear_modulus,
        unsigned int degree_l,
        double G_to_use,
        ssize_t num_ys, 
        double complex* starting_conditions_ptr,
        cpp_bool run_y_checks = True
        ) noexcept nogil:

    cdef unsigned char num_sols_for_assumption
    cdef unsigned char num_ys_for_assumption

    # Assume we are successful and adjust if we are not
    success_ptr[0] = True

    # For static liquid layers, no matter the other assumptions, we use saito's method.
    if (not (layer_type == 0)) and is_static:
        # Liquid Static Layer
        if run_y_checks:
            num_sols_for_assumption = 1
            num_ys_for_assumption   = 2
            if num_ys_for_assumption != num_ys:
                success_ptr[0] = False
                strcpy(message_ptr, 'RadialSolver::Shooting::FindStartingConditions: Incorrect number of ys for given the starting condition assumptions.')
        if success_ptr[0]:
            cf_saito_liquid_static_inccompressible(
                radius, degree_l, num_ys, starting_conditions_ptr
                )
        
    # Work through the Kamata models
    elif use_kamata:
        # Kamata solid layer
        if (layer_type == 0):
            # Solid layer
            if is_static and is_incompressible:
                success_ptr[0] = False
                strcpy(message_ptr, 'RadialSolver::Shooting::FindStartingConditions: Incompressibility is not implemented for Kamata starting conditions for static-solid layers.\nReccomend using dynamic-incompressible instead.')
            elif is_static and (not is_incompressible):
                if run_y_checks:  
                    num_sols_for_assumption = 3
                    num_ys_for_assumption   = 6
                    if num_ys_for_assumption != num_ys:
                        success_ptr[0] = False
                        strcpy(message_ptr, 'RadialSolver::Shooting::FindStartingConditions: Incorrect number of ys for given the starting condition assumptions.')
                if success_ptr[0]:
                    cf_kamata_solid_static_compressible(
                        radius, density, bulk_modulus, shear_modulus, degree_l, G_to_use, num_ys, starting_conditions_ptr
                    )
            elif (not is_static) and is_incompressible:
                if run_y_checks:
                    num_sols_for_assumption = 3
                    num_ys_for_assumption   = 6
                    if num_ys_for_assumption != num_ys:
                        success_ptr[0] = False
                        strcpy(message_ptr, 'RadialSolver::Shooting::FindStartingConditions: Incorrect number of ys for given the starting condition assumptions.')
                if success_ptr[0]:
                    cf_kamata_solid_dynamic_incompressible(
                        frequency, radius, density, shear_modulus, degree_l, G_to_use, num_ys, starting_conditions_ptr
                        )
            else:
                if run_y_checks:
                    num_sols_for_assumption = 3
                    num_ys_for_assumption   = 6
                    if num_ys_for_assumption != num_ys:
                        success_ptr[0] = False
                        strcpy(message_ptr, 'RadialSolver::Shooting::FindStartingConditions: Incorrect number of ys for given the starting condition assumptions.')
                if success_ptr[0]:
                    cf_kamata_solid_dynamic_compressible(
                        frequency, radius, density, bulk_modulus, shear_modulus, degree_l, G_to_use, num_ys, 
                        starting_conditions_ptr
                        )
        else:
            if is_static and is_incompressible:
                # Covered by Saito method.
                pass
            elif is_static and (not is_incompressible):     
                # Covered by Saito method.
                pass
            elif (not is_static) and is_incompressible:
                if run_y_checks:
                    num_sols_for_assumption = 2
                    num_ys_for_assumption   = 4
                    if num_ys_for_assumption != num_ys:
                        success_ptr[0] = False
                        strcpy(message_ptr, 'RadialSolver::Shooting::FindStartingConditions: Incorrect number of ys for given the starting condition assumptions.')
                if success_ptr[0]:
                    cf_kamata_liquid_dynamic_incompressible(
                        frequency, radius, density, degree_l, G_to_use, num_ys, starting_conditions_ptr
                        )
                
            else:
                if run_y_checks:
                    num_sols_for_assumption = 2
                    num_ys_for_assumption   = 4
                    if num_ys_for_assumption != num_ys:
                        success_ptr[0] = False
                        strcpy(message_ptr, 'RadialSolver::Shooting::FindStartingConditions: Incorrect number of ys for given the starting condition assumptions.')
                if success_ptr[0]:
                    cf_kamata_liquid_dynamic_compressible(
                        frequency, radius, density, bulk_modulus, degree_l, G_to_use, num_ys, starting_conditions_ptr
                        )
                
    # Work through the Takeuchi models
    else:
        if is_incompressible:
            success_ptr[0] = False
            strcpy(message_ptr, 'RadialSolver::Shooting::FindStartingConditions: Incompressibility is not implemented for most of the Takeuchi starting conditions. \nReccomend using Kamata (set use_kamata=True) instead.')
        if (layer_type == 0):
            # Solid layer
            if is_static:
                if run_y_checks:
                    num_sols_for_assumption = 3
                    num_ys_for_assumption   = 6
                    if num_ys_for_assumption != num_ys:
                        success_ptr[0] = False
                        strcpy(message_ptr, 'RadialSolver::Shooting::FindStartingConditions: Incorrect number of ys for given the starting condition assumptions.')
                if success_ptr[0]:
                    cf_takeuchi_solid_static_compressible(
                        radius, density, bulk_modulus, shear_modulus, degree_l, G_to_use, num_ys, starting_conditions_ptr
                    )
            else:
                if run_y_checks:
                    num_sols_for_assumption = 3
                    num_ys_for_assumption   = 6
                    if num_ys_for_assumption != num_ys:
                        success_ptr[0] = False
                        strcpy(message_ptr, 'RadialSolver::Shooting::FindStartingConditions:: Incorrect number of ys for given the starting condition assumptions.')
                if success_ptr[0]:
                    cf_takeuchi_solid_dynamic_compressible(
                        frequency, radius, density, bulk_modulus, shear_modulus,
                        degree_l, G_to_use, num_ys, starting_conditions_ptr
                        )
        else:
            if is_static:
                # Handled by Saito
                pass
            else:
                if run_y_checks:
                    num_sols_for_assumption = 2
                    num_ys_for_assumption   = 4
                    if num_ys_for_assumption != num_ys:
                        success_ptr[0] = False
                        strcpy(message_ptr, 'RadialSolver::Shooting::FindStartingConditions: Incorrect number of ys for given the starting condition assumptions.')
                if success_ptr[0]:
                    cf_takeuchi_liquid_dynamic_compressible(
                        frequency, radius, density, bulk_modulus, degree_l, G_to_use, num_ys, starting_conditions_ptr
                        )
    


cdef size_t cf_find_num_shooting_solutions(
        int layer_type,
        bint is_static,
        bint is_incompressible
        ) noexcept nogil:

    # Initialize
    cdef Py_ssize_t num_sols
    num_sols = 0

    if (layer_type == 0):
        # Solid
        if is_static:
            if is_incompressible:
                # TODO: Confirm
                num_sols = 3
            else:
                num_sols = 3
        else:
            # Dynamic
            if is_incompressible:
                # TODO: Confirm
                num_sols = 3
            else:
                num_sols = 3
    else:
        # Liquid
        if is_static:
            if is_incompressible:
                # TODO: Confirm
                num_sols = 1
            else:
                num_sols = 1
        else:
            # Dynamic
            if is_incompressible:
                # TODO: Confirm
                num_sols = 2
            else:
                num_sols = 2
    return num_sols

cdef void cf_solve_upper_y_at_interface(
        double complex* lower_layer_y_ptr,
        double complex* upper_layer_y_ptr,
        size_t num_sols_lower,
        size_t num_sols_upper,
        size_t max_num_y,
        int lower_layer_type,
        bint lower_is_static,
        bint lower_is_incompressible,
        int upper_layer_type,
        bint upper_is_static,
        bint upper_is_incompressible,
        double interface_gravity,
        double liquid_density,
        double G_to_use
        ) noexcept nogil:

    cdef size_t yi_lower, yi_upper, soli_lower, soli_upper

    cdef bint upper_solid = False
    cdef bint lower_solid = False
    if lower_layer_type == 0:
        lower_solid = True
    if upper_layer_type == 0:
        upper_solid = True

    cdef bint solid_solid, solid_liquid, liquid_solid, liquid_liquid
    solid_solid   = lower_solid and upper_solid
    solid_liquid  = lower_solid and not upper_solid
    liquid_solid  = not lower_solid and upper_solid
    liquid_liquid = not lower_solid and not upper_solid

    cdef bint static_static, static_dynamic, dynamic_static, dynamic_dynamic
    static_static   = lower_is_static and upper_is_static
    static_dynamic  = lower_is_static and not upper_is_static
    dynamic_static  = not lower_is_static and upper_is_static
    dynamic_dynamic = not lower_is_static and not upper_is_static

    cdef bint compress_compress, compress_incompress, incompress_compress, incompress_incompress
    compress_compress     = not lower_is_incompressible and not upper_is_incompressible
    compress_incompress   = not lower_is_incompressible and upper_is_incompressible
    incompress_compress   = lower_is_incompressible and not upper_is_incompressible
    incompress_incompress = lower_is_incompressible and upper_is_incompressible
    # TODO: Compressibility is not currently taken into account. This needs to be checked asap!

    # Other constants that may be needed
    cdef double complex lambda_1 = cmplx_NAN
    cdef double complex lambda_2 = cmplx_NAN
    cdef double complex coeff_1  = cmplx_NAN
    cdef double complex coeff_2  = cmplx_NAN
    cdef double complex coeff_3  = cmplx_NAN
    cdef double complex coeff_4  = cmplx_NAN
    cdef double complex coeff_5  = cmplx_NAN
    cdef double complex coeff_6  = cmplx_NAN
    cdef double complex frac_1   = cmplx_NAN
    cdef double complex frac_2   = cmplx_NAN
    cdef double complex const_1  = cmplx_NAN

    cdef double g_const = 4. * pi * G_to_use

    # Initialize upper y to nan
    for yi_upper in range(18):
        upper_layer_y_ptr[yi_upper] = cmplx_NAN

    if solid_solid:
        # if static_static or static_dynamic or dynamic_static or dynamic_dynamic:
        # Does not matter if the layers are static or dynamic, solid-solid exchange perfectly.
        # All solutions carry over
        for yi_lower in range(max_num_y):
            # They have same number of ys
            yi_upper = yi_lower
            for soli_lower in range(num_sols_lower):
                # They have same number of sols
                soli_upper = soli_lower
                upper_layer_y_ptr[soli_upper * max_num_y + yi_upper] = \
                    lower_layer_y_ptr[soli_lower * max_num_y + yi_lower]
    elif liquid_liquid:
        # Liquid interfaces carry everything over if they are both static or both dynamic.
        if static_static or dynamic_dynamic:
            # All solutions carry over
            for yi_lower in range(max_num_y):
                # They have same number of ys
                yi_upper = yi_lower
                for soli_lower in range(num_sols_lower):
                    # They have same number of sols
                    soli_upper = soli_lower
                    upper_layer_y_ptr[soli_upper * max_num_y + yi_upper] = \
                        lower_layer_y_ptr[soli_lower * max_num_y + yi_lower]
        elif static_dynamic:
            # For an upper layer that is liquid and dynamic there will be two independent solutions that need an initial guess.
            # # Solution 1
            # y_1_dynamic = 0
            upper_layer_y_ptr[0] = cmplx_zero
            # y_2_dynamic = -rho * y_5_static
            upper_layer_y_ptr[1] = -liquid_density * lower_layer_y_ptr[0]
            # y_5_dynamic = y_5_static
            upper_layer_y_ptr[2] = lower_layer_y_ptr[0]
            # y_6_dynamic = y_7_static + (4 pi G rho / g) * y_5_static
            upper_layer_y_ptr[3] = \
                lower_layer_y_ptr[1] + (g_const * liquid_density / interface_gravity) * lower_layer_y_ptr[0]
            
            # # Solution 2
            # y_1_dynamic = 1.
            upper_layer_y_ptr[1 * max_num_y + 0] = cf_build_dblcmplx(1., 0.)
            # y_2_dynamic = rho * g * y_1_dynamic
            upper_layer_y_ptr[1 * max_num_y + 1] = \
                liquid_density * interface_gravity * upper_layer_y_ptr[1 * max_num_y + 0]
            # y_5_dynamic = 0.
            upper_layer_y_ptr[1 * max_num_y + 2] = cmplx_zero
            # y_6_dynamic = -4 pi G rho y_1_dynamic
            upper_layer_y_ptr[1 * max_num_y + 3] = \
                -g_const * liquid_density * upper_layer_y_ptr[1 * max_num_y + 0]
        elif dynamic_static:
            # lambda_j = (y_2j - rho * ( g * y_1j - y_5j))
            lambda_1 = \
                lower_layer_y_ptr[1] - \
                liquid_density * (interface_gravity * lower_layer_y_ptr[0] - lower_layer_y_ptr[2])
            lambda_2 = \
                lower_layer_y_ptr[1 * max_num_y + 1] - \
                liquid_density * (interface_gravity * lower_layer_y_ptr[1 * max_num_y + 0] -
                                  lower_layer_y_ptr[1 * max_num_y + 2])

            # Set the first coefficient to 1. It will be solved for later on during the collapse phase.
            coeff_1 = cf_build_dblcmplx(1., 0.)
            # The other coefficient which is related to 1 via...
            coeff_2 = -(lambda_1 / lambda_2) * coeff_1

            coeff_3 = lower_layer_y_ptr[3] + g_const * lower_layer_y_ptr[1] / interface_gravity
            coeff_4 = \
                lower_layer_y_ptr[1 * max_num_y + 3] + \
                g_const * lower_layer_y_ptr[1 * max_num_y + 1] / interface_gravity

            # y^liq(st)_5 = C^liq(dy)_1 * y^liq(dy)_5,1 + C^liq(dy)_2 * y^liq(dy)_5,2
            upper_layer_y_ptr[0] = coeff_1 * lower_layer_y_ptr[2] + coeff_2 * lower_layer_y_ptr[1 * max_num_y + 2]
            # y^liq(st)_7 = C^liq(dy)_1 * y^liq(dy)_7,1 + C^liq(dy)_2 * y^liq(dy)_7,2
            upper_layer_y_ptr[1] = coeff_1 * coeff_3 + coeff_2 * coeff_4
        else:
            # How did you get here...
            pass
    elif liquid_solid:
        if dynamic_dynamic or dynamic_static:
            # As far as I am aware, the dynamic_static and dynamic_dynamic solutions are the same.

            # See Eqs. 148-149 in TS72
            # For a dynamic solid layer there will be three independent solutions that we need an initial guess for.
            for soli_upper in range(3):
                if soli_upper == 0 or soli_upper == 1:
                    # For a dynamic liquid layer there will be two independent solutions at the top of the layer
                    upper_layer_y_ptr[soli_upper * max_num_y + 0] = lower_layer_y_ptr[soli_upper * max_num_y + 0]
                    upper_layer_y_ptr[soli_upper * max_num_y + 1] = lower_layer_y_ptr[soli_upper * max_num_y + 1]
                    upper_layer_y_ptr[soli_upper * max_num_y + 4] = lower_layer_y_ptr[soli_upper * max_num_y + 2]
                    upper_layer_y_ptr[soli_upper * max_num_y + 5] = lower_layer_y_ptr[soli_upper * max_num_y + 3]

                    # For solutions 1 and 2: y_3 and y_4 for the solid layer are zero
                    upper_layer_y_ptr[soli_upper * max_num_y + 2] = cmplx_zero
                    upper_layer_y_ptr[soli_upper * max_num_y + 3] = cmplx_zero
                else:
                    # For the third solid solution all the y's are set to zero except y_3.
                    upper_layer_y_ptr[soli_upper * max_num_y + 0] = cmplx_zero
                    upper_layer_y_ptr[soli_upper * max_num_y + 1] = cmplx_zero
                    upper_layer_y_ptr[soli_upper * max_num_y + 2] = cf_build_dblcmplx(1., 0.)
                    upper_layer_y_ptr[soli_upper * max_num_y + 3] = cmplx_zero
                    upper_layer_y_ptr[soli_upper * max_num_y + 4] = cmplx_zero
                    upper_layer_y_ptr[soli_upper * max_num_y + 5] = cmplx_zero
        elif static_dynamic or static_static:
            # As far as I am aware, the static_static and static_dynamic solutions are the same.

            # Eqs. 20 in S74
            # For a static liquid layer there will be one independent solutions at the top of the layer
            # We need to solve for six solutions in the static or dynamic solid upper layer
            upper_layer_y_ptr[0] = cmplx_zero
            # y_2_sol = -rho * y_5_liq
            upper_layer_y_ptr[1] = -liquid_density * lower_layer_y_ptr[0]
            upper_layer_y_ptr[2] = cmplx_zero
            upper_layer_y_ptr[3] = cmplx_zero
            # y_5_sol = y_5_liq
            upper_layer_y_ptr[4] = lower_layer_y_ptr[0]
            # y_6_sol = y_7_liq + (4 pi G rho / g) y_5_liq
            upper_layer_y_ptr[5] = \
                lower_layer_y_ptr[1] + (g_const * liquid_density / interface_gravity) * lower_layer_y_ptr[0]
            # y_1_sol = 1.
            upper_layer_y_ptr[1 * max_num_y + 0] = cf_build_dblcmplx(1., 0.)
            # y_2_sol = rho * g * y_1_sol
            upper_layer_y_ptr[1 * max_num_y + 1] = \
                liquid_density * interface_gravity * upper_layer_y_ptr[1 * max_num_y + 0]
            upper_layer_y_ptr[1 * max_num_y + 2] = cmplx_zero
            upper_layer_y_ptr[1 * max_num_y + 3] = cmplx_zero
            upper_layer_y_ptr[1 * max_num_y + 4] = cmplx_zero
            # y_6_sol = -4 pi G rho y_1_sol
            upper_layer_y_ptr[1 * max_num_y + 5] = \
                -g_const * liquid_density * upper_layer_y_ptr[1 * max_num_y + 0]

            upper_layer_y_ptr[2 * max_num_y + 0] = cmplx_zero
            upper_layer_y_ptr[2 * max_num_y + 1] = cmplx_zero
            # y_3_sol = 1.
            upper_layer_y_ptr[2 * max_num_y + 2] = cf_build_dblcmplx(1., 0.)
            upper_layer_y_ptr[2 * max_num_y + 3] = cmplx_zero
            upper_layer_y_ptr[2 * max_num_y + 4] = cmplx_zero
            upper_layer_y_ptr[2 * max_num_y + 5] = cmplx_zero
    elif solid_liquid:

        if dynamic_dynamic or static_dynamic:
            # As far as I am aware, static_dynamic and dynamic_dynamic should be the same.
            # Eqs. 140-144 in TS72
            # Three solutions in the lower solid layer and two in the upper liquid.

            for soli_upper in range(2):
                # Solve for y^liq_1, y^liq_2, y^liq_5, y^liq_6 (TS72 Eq. 143)
                #    Note that the liquid solution does not have y_3, y_4 which are index 2, 3 for solid solution.
                coeff_1 = lower_layer_y_ptr[soli_upper * max_num_y + 3] / lower_layer_y_ptr[2 * max_num_y + 3]

                upper_layer_y_ptr[soli_upper * max_num_y + 0] = \
                    lower_layer_y_ptr[soli_upper * max_num_y + 0] - coeff_1 * lower_layer_y_ptr[2 * max_num_y + 0]
                upper_layer_y_ptr[soli_upper * max_num_y + 1] = \
                    lower_layer_y_ptr[soli_upper * max_num_y + 1] - coeff_1 * lower_layer_y_ptr[2 * max_num_y + 1]
                upper_layer_y_ptr[soli_upper * max_num_y + 2] = \
                    lower_layer_y_ptr[soli_upper * max_num_y + 4] - coeff_1 * lower_layer_y_ptr[2 * max_num_y + 4]
                upper_layer_y_ptr[soli_upper * max_num_y + 3] = \
                    lower_layer_y_ptr[soli_upper * max_num_y + 5] - coeff_1 * lower_layer_y_ptr[2 * max_num_y + 5]
        elif dynamic_static or static_static:
            # As far as I am aware, static_static and dynamic_static should work the same.
            # Eq. 21 in S74
            frac_1 = -lower_layer_y_ptr[0 * max_num_y + 3] / lower_layer_y_ptr[2 * max_num_y + 3]
            frac_2 = -lower_layer_y_ptr[1 * max_num_y + 3] / lower_layer_y_ptr[2 * max_num_y + 3]

            # lambda_j = (y_2j + f_j y_23) - rho( g(y_1j + f_j y_13) - (y_5j + f_j y_53))
            lambda_1 = \
                lower_layer_y_ptr[1] + frac_1 * lower_layer_y_ptr[2 * max_num_y + 1] - \
                liquid_density * (
                        interface_gravity * (
                            lower_layer_y_ptr[0] + frac_1 * lower_layer_y_ptr[2 * max_num_y + 0]) -
                        (lower_layer_y_ptr[4] + frac_1 * lower_layer_y_ptr[2 * max_num_y + 4])
                )
            lambda_2 = \
                lower_layer_y_ptr[1 * max_num_y + 1] + frac_2 * lower_layer_y_ptr[2 * max_num_y + 1] - \
                liquid_density * (
                        interface_gravity * (
                            lower_layer_y_ptr[1 * max_num_y + 0] + frac_2 * lower_layer_y_ptr[2 * max_num_y + 0]) -
                        (lower_layer_y_ptr[1 * max_num_y + 4] + frac_2 * lower_layer_y_ptr[2 * max_num_y + 4])
                )

            # Set the first coefficient to 1. It will be solved for later on during the collapse phase.
            coeff_1 = 1.
            # The other two coefficients are related to 1 via...
            coeff_2 = -(lambda_1 / lambda_2) * coeff_1
            coeff_3 = frac_1 * coeff_1 + frac_2 * coeff_2

            const_1 = (g_const / interface_gravity)

            coeff_4 = lower_layer_y_ptr[0 * max_num_y + 5] + const_1 * lower_layer_y_ptr[0 * max_num_y + 1]
            coeff_5 = lower_layer_y_ptr[1 * max_num_y + 5] + const_1 * lower_layer_y_ptr[1 * max_num_y + 1]
            coeff_6 = lower_layer_y_ptr[2 * max_num_y + 5] + const_1 * lower_layer_y_ptr[2 * max_num_y + 1]

            # y^liq_5 = C^sol_1 * y^sol_5,1 + C^sol_2 * y^sol_5,2 + C^sol_3 * y^sol_5,3
            upper_layer_y_ptr[0] = \
                coeff_1 * lower_layer_y_ptr[0 * max_num_y + 4] + \
                coeff_2 * lower_layer_y_ptr[1 * max_num_y + 4] + \
                coeff_3 * lower_layer_y_ptr[2 * max_num_y + 4]
            # y^liq_7 = C^sol_1 * y^sol_7,1 + C^sol_2 * y^sol_7,2 + C^sol_3 * y^sol_7,3
            upper_layer_y_ptr[1] = \
                coeff_1 * coeff_4 + coeff_2 * coeff_5 + coeff_3 * coeff_6




cdef void cf_top_to_bottom_interface_bc(
        double complex* constant_vector_ptr,
        double complex* layer_above_constant_vector_ptr,
        double complex* uppermost_y_per_solution_ptr,
        double gravity_upper,
        double layer_above_lower_gravity,
        double density_upper,
        double layer_above_lower_density,
        int layer_type,
        int layer_above_type,
        bint layer_is_static,
        bint layer_above_is_static,
        bint layer_is_incomp,
        bint layer_above_is_incomp,
        unsigned char num_sols,
        unsigned char max_num_y
        ) noexcept nogil:
    
    # Loop variables
    cdef unsigned char solution_i
    
    # Interfaces are defined at the bottom of the layer in question. However, this function is calculating
    #  the transition at the top of each layer as it works its way down.
    #  So, for interface values, we actually need the ones of the layer above us.
    cdef double interface_gravity = 0.5 * (gravity_upper + layer_above_lower_gravity)
    cdef double liquid_density_at_interface = NAN

    cdef bint layer_is_solid = False
    cdef bint layer_above_is_solid = False
    if layer_type == 0:
        layer_is_solid = True
    if layer_above_type == 0:
        layer_above_is_solid = True

    if not layer_is_solid:
        if layer_is_static:
            liquid_density_at_interface = density_upper
        elif not layer_above_is_solid and layer_above_is_static:
            liquid_density_at_interface = layer_above_lower_density
        else:
            liquid_density_at_interface = density_upper
    elif not layer_above_is_solid:
        liquid_density_at_interface = layer_above_lower_density

    # Solve for constant vector
    cdef double complex y4_frac_1  = cmplx_NAN
    cdef double complex y4_frac_2  = cmplx_NAN
    cdef double complex gamma_1    = cmplx_NAN
    cdef double complex gamma_2    = cmplx_NAN
    cdef double complex lambda_1   = cmplx_NAN
    cdef double complex lambda_2   = cmplx_NAN
    cdef double complex lower_s1y1 = cmplx_NAN
    cdef double complex lower_s1y2 = cmplx_NAN
    cdef double complex lower_s1y5 = cmplx_NAN
    cdef double complex lower_s1y6 = cmplx_NAN
    cdef double complex lower_s2y1 = cmplx_NAN
    cdef double complex lower_s2y2 = cmplx_NAN
    cdef double complex lower_s2y5 = cmplx_NAN
    cdef double complex lower_s2y6 = cmplx_NAN

    if layer_is_solid:
        # Solid Layer
        if layer_above_is_solid:
            # Both layers are solid. Constants are the same.
            solution_i = 0
            while num_sols > solution_i:
                constant_vector_ptr[solution_i] = layer_above_constant_vector_ptr[solution_i]
                solution_i += 1
        else:
            # Create some helper functions that will be needed
            y4_frac_1 = (
                    -uppermost_y_per_solution_ptr[0 * max_num_y + 3] /
                    uppermost_y_per_solution_ptr[2 * max_num_y + 3]
                )
            y4_frac_2 = (
                    -uppermost_y_per_solution_ptr[1 * max_num_y + 3] /
                    uppermost_y_per_solution_ptr[2 * max_num_y + 3]
                )

            if layer_above_is_static:
                # Need to find 3 solid constants from 1 liquid constant
                # S74, Page 131
                constant_vector_ptr[0] = layer_above_constant_vector_ptr[0]
                # Derived by JPR based on Eq 21 (2nd line) of S74
                gamma_1 = \
                    (
                        uppermost_y_per_solution_ptr[0 * max_num_y + 1] +
                        y4_frac_1 * uppermost_y_per_solution_ptr[2 * max_num_y + 1]
                    ) - \
                    (
                        liquid_density_at_interface *
                        (
                            interface_gravity * 
                            (
                                uppermost_y_per_solution_ptr[0 * max_num_y + 0] +
                                y4_frac_1 * uppermost_y_per_solution_ptr[2 * max_num_y + 0]
                            ) -
                            (
                                uppermost_y_per_solution_ptr[0 * max_num_y + 4] +
                                y4_frac_1 * uppermost_y_per_solution_ptr[2 * max_num_y + 4]
                            )
                        )
                    )
                gamma_2 = \
                    (
                        uppermost_y_per_solution_ptr[1 * max_num_y + 1] + 
                        y4_frac_2 * uppermost_y_per_solution_ptr[2 * max_num_y + 1]
                    ) - \
                    (
                        liquid_density_at_interface *
                        (
                            interface_gravity * 
                            (
                                uppermost_y_per_solution_ptr[1 * max_num_y + 0] +
                                y4_frac_2 * uppermost_y_per_solution_ptr[2 * max_num_y + 0]
                            ) - 
                            (
                                uppermost_y_per_solution_ptr[1 * max_num_y + 4] + 
                                y4_frac_2 * uppermost_y_per_solution_ptr[2 * max_num_y + 4]
                            )
                        )
                    )

                constant_vector_ptr[1] = (-gamma_1 / gamma_2) * constant_vector_ptr[0]
                # TS72, Eq. 142 (utilizes y_4 = 0)
                constant_vector_ptr[2] = y4_frac_1 * constant_vector_ptr[0] + y4_frac_2 * constant_vector_ptr[1]

            else:
                # Need to find 3 solid constants from 2 liquid constants
                # TS72, Eq. 144
                constant_vector_ptr[0] = layer_above_constant_vector_ptr[0]
                constant_vector_ptr[1] = layer_above_constant_vector_ptr[1]
                # TS72, Eq. 142 (utilizes y_4 = 0)
                constant_vector_ptr[2] = y4_frac_1 * constant_vector_ptr[0] + y4_frac_2 * constant_vector_ptr[1]
    else:
        if layer_is_static:
            if not layer_above_is_solid:
                # Liquid layer above
                if layer_above_is_static:
                    # Both layers are static liquids. Constants are the same.
                    constant_vector_ptr[0] = layer_above_constant_vector_ptr[0]
                else:
                    # Dynamic liquid above
                    # JPR decided to follow a similar approach as Eq. 20 in S74:
                    #   Treat the lower static liquid as normal.
                    #   The upper dynamic liquid layer is treated like the solid layer in Eq. 20 except
                    #    that y_3 is undefined as is "set 3" solution mentioned in that text.
                    constant_vector_ptr[0] = layer_above_constant_vector_ptr[0]
            else:
                # Solid layer above
                # Based on S74. The constant in this layer is just equal to the constant in solution 1 of the
                #  layer above.
                constant_vector_ptr[0] = layer_above_constant_vector_ptr[0]
        else:
            if not layer_above_is_solid:
                # Liquid layer above
                if layer_above_is_static:
                    # Need to find 2 liquid (dynamic) constants from 1 liquid (static) constant
                    # S74, Page 131
                    constant_vector_ptr[0] = layer_above_constant_vector_ptr[0]
                    # Derived by JPR based on Eq 21 (2nd line) of S74
                    # Pull out ys
                    # # Solution 1
                    lower_s1y1 = uppermost_y_per_solution_ptr[0 * max_num_y + 0]
                    lower_s1y2 = uppermost_y_per_solution_ptr[0 * max_num_y + 1]
                    lower_s1y5 = uppermost_y_per_solution_ptr[0 * max_num_y + 2]
                    lower_s1y6 = uppermost_y_per_solution_ptr[0 * max_num_y + 3]
                    # # Solution 2
                    lower_s2y1 = uppermost_y_per_solution_ptr[1 * max_num_y + 0]
                    lower_s2y2 = uppermost_y_per_solution_ptr[1 * max_num_y + 1]
                    lower_s2y5 = uppermost_y_per_solution_ptr[1 * max_num_y + 2]
                    lower_s2y6 = uppermost_y_per_solution_ptr[1 * max_num_y + 3]
                    # lambda_j = (y_2j - rho * ( g * y_1j - y_5j))
                    lambda_1 = lower_s1y2 - liquid_density_at_interface * \
                            (interface_gravity * lower_s1y1 - lower_s1y5)
                    lambda_2 = lower_s2y2 - liquid_density_at_interface * \
                            (interface_gravity * lower_s2y1 - lower_s2y5)
                    constant_vector_ptr[1] = (-lambda_1 / lambda_2) * constant_vector_ptr[0]
                else:
                    # Both layers are dynamic liquids. Constants are the same.
                    solution_i = 0
                    while num_sols > solution_i:
                        constant_vector_ptr[solution_i] = layer_above_constant_vector_ptr[solution_i]
                        solution_i += 1
            else:
                # Solid layer above
                # TS72 Eqs. 148-149
                constant_vector_ptr[0] = layer_above_constant_vector_ptr[0]
                constant_vector_ptr[1] = layer_above_constant_vector_ptr[1]

cdef void cf_solid_dynamic_compressible(
        double* dy_ptr,
        double radius,
        double* y_ptr,
        const void* void_args_ptr,
        PreEvalFunc eos_func) noexcept nogil:
    """

    References
    ----------
    KMN15; B15; TS72
    """

    # Recast the additional arguments
    cdef RadialSolverDiffeqArgStruct* args_ptr = <RadialSolverDiffeqArgStruct*>void_args_ptr

    # Update Equation of State at this radius value
    cdef double[7] eos_array 
    cdef double* eos_array_ptr = &eos_array[0]
    # Call equation of state solution for this layer.
    args_ptr.eos_solution_ptr.call(radius, eos_array_ptr)
    # Pull out results.
    # The EOS stores 7 doubles:
    #   0: Gravity
    #   1: Pressure
    #   2: Density
    #   3: Shear Mod (real)
    #   4: Shear Mod (imag)
    #   5: Bulk Mod (real)
    #   6: Bulk Mod (imag)
    cdef double gravity                  = eos_array_ptr[0]
    cdef double density                  = eos_array_ptr[2]
    cdef double complex shear_modulus    = cf_build_dblcmplx(eos_array_ptr[3], eos_array_ptr[4])
    cdef double complex bulk_modulus     = cf_build_dblcmplx(eos_array_ptr[5], eos_array_ptr[6])

    # Pull out y values
    cdef double complex y1 = cf_build_dblcmplx(y_ptr[0], y_ptr[1])
    cdef double complex y2 = cf_build_dblcmplx(y_ptr[2], y_ptr[3])
    cdef double complex y3 = cf_build_dblcmplx(y_ptr[4], y_ptr[5])
    cdef double complex y4 = cf_build_dblcmplx(y_ptr[6], y_ptr[7])
    cdef double complex y5 = cf_build_dblcmplx(y_ptr[8], y_ptr[9])
    cdef double complex y6 = cf_build_dblcmplx(y_ptr[10], y_ptr[11])

    # Convert compressibility parameters (the first lame parameter can be complex)
    cdef double complex lame = (bulk_modulus - (2. / 3.) * shear_modulus)

    # Optimizations
    cdef double r_inverse                = 1. / radius
    cdef double density_gravity          = density * gravity
    cdef double dynamic_term             = -args_ptr.frequency * args_ptr.frequency * density * radius
    cdef double grav_term                = args_ptr.grav_coeff * density
    cdef double complex lame_2mu         = lame + 2. * shear_modulus
    cdef double complex lame_2mu_inverse = 1. / lame_2mu
    cdef double complex two_shear_r_inv  = 2. * shear_modulus * r_inverse
    cdef double complex y1_y3_term       = 2. * y1 - args_ptr.llp1 * y3

    # See Eq. 82 in TS72 or Eqs. 4--9 in KMN15 or Eqs. 13--18 in B15
    #   Note: There appears to be a missing factor of mu^2 in some of the terms in KMN15.
    # dy2 and dy4 contain all three of: dynamic, viscoelastic, and gravitational terms.
    cdef double complex dy1 = \
        lame_2mu_inverse * (
            y1_y3_term * -lame * r_inverse +
            y2
        )

    cdef double complex dy2 = \
        r_inverse * (
            y1 * (dynamic_term - 2. * density_gravity) +
            y2 * -2. +
            y4 * args_ptr.llp1 +
            y5 * density * args_ptr.lp1 +
            y6 * -density * radius +
            dy1 * 2. * lame +
            y1_y3_term * (2. * (lame + shear_modulus) * r_inverse - density_gravity)
        )

    cdef double complex dy3 = \
        y1 * -r_inverse + \
        y3 * r_inverse + \
        y4 * (1. / shear_modulus)

    cdef double complex dy4 = \
        r_inverse * (
            y1 * (density_gravity + two_shear_r_inv) +
            y3 * (dynamic_term - two_shear_r_inv) +
            y4 * -3. +
            y5 * -density +
            dy1 * -lame +
            y1_y3_term * -lame_2mu * r_inverse
        )

    cdef double complex dy5 = \
        y1 * grav_term + \
        y5 * -args_ptr.lp1 * r_inverse + \
        y6

    cdef double complex dy6 = \
        r_inverse * (
            y1 * grav_term * args_ptr.lm1 +
            y6 * args_ptr.lm1 +
            y1_y3_term * grav_term
        )

    # Convert back to floats
    dy_ptr[0]  = dy1.real
    dy_ptr[1]  = dy1.imag
    dy_ptr[2]  = dy2.real
    dy_ptr[3]  = dy2.imag
    dy_ptr[4]  = dy3.real
    dy_ptr[5]  = dy3.imag
    dy_ptr[6]  = dy4.real
    dy_ptr[7]  = dy4.imag
    dy_ptr[8]  = dy5.real
    dy_ptr[9]  = dy5.imag
    dy_ptr[10] = dy6.real
    dy_ptr[11] = dy6.imag


cdef void cf_solid_dynamic_incompressible(
        double* dy_ptr,
        double radius,
        double* y_ptr,
        const void* void_args_ptr,
        PreEvalFunc eos_func) noexcept nogil:
    """

    References
    ----------
    KMN15; B15; TS72
    """

    # Recast the additional arguments
    cdef RadialSolverDiffeqArgStruct* args_ptr = <RadialSolverDiffeqArgStruct*>void_args_ptr

    # Update Equation of State at this radius value
    cdef double[7] eos_array 
    cdef double* eos_array_ptr = &eos_array[0]
    # Call equation of state solution for this layer.
    args_ptr.eos_solution_ptr.call(radius, eos_array_ptr)
    # Pull out results.
    # The EOS stores 7 doubles:
    #   0: Gravity
    #   1: Pressure
    #   2: Density
    #   3: Shear Mod (real)
    #   4: Shear Mod (imag)
    #   5: Bulk Mod (real)
    #   6: Bulk Mod (imag)
    cdef double gravity                  = eos_array_ptr[0]
    cdef double density                  = eos_array_ptr[2]
    cdef double complex shear_modulus    = cf_build_dblcmplx(eos_array_ptr[3], eos_array_ptr[4])
    cdef double complex bulk_modulus     = cf_build_dblcmplx(eos_array_ptr[5], eos_array_ptr[6])

    # Pull out y values
    cdef double complex y1 = cf_build_dblcmplx(y_ptr[0], y_ptr[1])
    cdef double complex y2 = cf_build_dblcmplx(y_ptr[2], y_ptr[3])
    cdef double complex y3 = cf_build_dblcmplx(y_ptr[4], y_ptr[5])
    cdef double complex y4 = cf_build_dblcmplx(y_ptr[6], y_ptr[7])
    cdef double complex y5 = cf_build_dblcmplx(y_ptr[8], y_ptr[9])
    cdef double complex y6 = cf_build_dblcmplx(y_ptr[10], y_ptr[11])

    # Optimizations
    cdef double r_inverse               = 1. / radius
    cdef double density_gravity         = density * gravity
    cdef double dynamic_term            = -args_ptr.frequency * args_ptr.frequency * density * radius
    cdef double grav_term               = args_ptr.grav_coeff * density
    cdef double complex two_shear_r_inv = 2. * shear_modulus * r_inverse
    cdef double complex y1_y3_term      = 2. * y1 - args_ptr.llp1 * y3

    # See Eq. 82 in TS72 or Eqs. 4--9 in KMN15 or Eqs. 13--18 in B15
    #   Note: There appears to be a missing factor of mu^2 in some of the terms in KMN15.
    # dy2 and dy4 contain all three of: dynamic, viscoelastic, and gravitational terms.
    cdef double complex dy1 = \
        y1_y3_term * -1. * r_inverse

    cdef double complex dy2 = \
        r_inverse * (
            y1 * (dynamic_term + 12. * shear_modulus * r_inverse - 4. * density_gravity) +
            y3 * args_ptr.llp1 * (density_gravity - 6. * shear_modulus * r_inverse) +
            y4 * args_ptr.llp1 +
            y5 * density * args_ptr.lp1 +
            y6 * -density * radius
        )

    cdef double complex dy3 = \
        y1 * -r_inverse + \
        y3 * r_inverse + \
        y4 * (1. / shear_modulus)

    cdef double complex dy4 = \
        r_inverse * (
            y1 * (density_gravity - 3. * two_shear_r_inv) +
            y2 * -1. +
            y3 * (dynamic_term + two_shear_r_inv * (2. * args_ptr.llp1 - 1.)) +
            y4 * -3. +
            y5 * -density
        )

    cdef double complex dy5 = \
        y1 * grav_term + \
        y5 * -args_ptr.lp1 * r_inverse + \
        y6

    cdef double complex dy6 = \
        r_inverse * (
            y1 * grav_term * args_ptr.lm1 +
            y6 * args_ptr.lm1 +
            y1_y3_term * grav_term
        )

    # Convert back to floats
    dy_ptr[0]  = dy1.real
    dy_ptr[1]  = dy1.imag
    dy_ptr[2]  = dy2.real
    dy_ptr[3]  = dy2.imag
    dy_ptr[4]  = dy3.real
    dy_ptr[5]  = dy3.imag
    dy_ptr[6]  = dy4.real
    dy_ptr[7]  = dy4.imag
    dy_ptr[8]  = dy5.real
    dy_ptr[9]  = dy5.imag
    dy_ptr[10] = dy6.real
    dy_ptr[11] = dy6.imag


cdef void cf_solid_static_compressible(
        double* dy_ptr,
        double radius,
        double* y_ptr,
        const void* void_args_ptr,
        PreEvalFunc eos_func) noexcept nogil:

    # Recast the additional arguments
    cdef RadialSolverDiffeqArgStruct* args_ptr = <RadialSolverDiffeqArgStruct*>void_args_ptr

    # Update Equation of State at this radius value
    cdef double[7] eos_array 
    cdef double* eos_array_ptr = &eos_array[0]
    # Call equation of state solution for this layer.
    args_ptr.eos_solution_ptr.call(radius, eos_array_ptr)
    # Pull out results.
    # The EOS stores 7 doubles:
    #   0: Gravity
    #   1: Pressure
    #   2: Density
    #   3: Shear Mod (real)
    #   4: Shear Mod (imag)
    #   5: Bulk Mod (real)
    #   6: Bulk Mod (imag)
    cdef double gravity                  = eos_array_ptr[0]
    cdef double density                  = eos_array_ptr[2]
    cdef double complex shear_modulus    = cf_build_dblcmplx(eos_array_ptr[3], eos_array_ptr[4])
    cdef double complex bulk_modulus     = cf_build_dblcmplx(eos_array_ptr[5], eos_array_ptr[6])

    # Pull out y values
    cdef double complex y1 = cf_build_dblcmplx(y_ptr[0], y_ptr[1])
    cdef double complex y2 = cf_build_dblcmplx(y_ptr[2], y_ptr[3])
    cdef double complex y3 = cf_build_dblcmplx(y_ptr[4], y_ptr[5])
    cdef double complex y4 = cf_build_dblcmplx(y_ptr[6], y_ptr[7])
    cdef double complex y5 = cf_build_dblcmplx(y_ptr[8], y_ptr[9])
    cdef double complex y6 = cf_build_dblcmplx(y_ptr[10], y_ptr[11])

    # Convert compressibility parameters (the first lame parameter can be complex)
    cdef double complex lame = (bulk_modulus - (2. / 3.) * shear_modulus)

    # Optimizations
    cdef double r_inverse                = 1. / radius
    cdef double density_gravity          = density * gravity
    cdef double grav_term                = args_ptr.grav_coeff * density
    cdef double complex lame_2mu         = lame + 2. * shear_modulus
    cdef double complex lame_2mu_inverse = 1. / lame_2mu
    cdef double complex two_shear_r_inv  = 2. * shear_modulus * r_inverse
    cdef double complex y1_y3_term       = 2. * y1 - args_ptr.llp1 * y3

    # See Eq. 82 in TS72 or Eqs. 4--9 in KMN15 or Eqs. 13--18 in B15
    #   Note: There appears to be a missing factor of mu^2 in some of the terms in KMN15.
    # The static case just sets all frequency_to_use dependence in these equations to zero.
    # dy2 and dy4 contain: viscoelastic, and gravitational terms.
    cdef double complex dy1 = \
        lame_2mu_inverse * (
            y1_y3_term * -lame * r_inverse +
            y2
        )

    cdef double complex dy2 = \
        r_inverse * (
            y1 * -2. * density_gravity +
            y2 * -2. +
            y4 * args_ptr.llp1 +
            y5 * density * args_ptr.lp1 +
            y6 * -density * radius +
            dy1 * 2. * lame +
            y1_y3_term * (2. * (lame + shear_modulus) * r_inverse - density_gravity)
        )

    cdef double complex dy3 = \
        y1 * -r_inverse + \
        y3 * r_inverse + \
        y4 * (1. / shear_modulus)

    cdef double complex dy4 = \
        r_inverse * (
            y1 * (density_gravity + two_shear_r_inv) +
            y3 * -two_shear_r_inv +
            y4 * -3. +
            y5 * -density +
            dy1 * -lame +
            y1_y3_term * -lame_2mu * r_inverse
        )

    cdef double complex dy5 = \
        y1 * grav_term + \
        y5 * -args_ptr.lp1 * r_inverse + \
        y6

    cdef double complex dy6 = \
        r_inverse * (
            y1 * grav_term * args_ptr.lm1 +
            y6 * args_ptr.lm1 +
            y1_y3_term * grav_term
        )

    # Convert back to floats
    dy_ptr[0]  = dy1.real
    dy_ptr[1]  = dy1.imag
    dy_ptr[2]  = dy2.real
    dy_ptr[3]  = dy2.imag
    dy_ptr[4]  = dy3.real
    dy_ptr[5]  = dy3.imag
    dy_ptr[6]  = dy4.real
    dy_ptr[7]  = dy4.imag
    dy_ptr[8]  = dy5.real
    dy_ptr[9]  = dy5.imag
    dy_ptr[10] = dy6.real
    dy_ptr[11] = dy6.imag


cdef void cf_solid_static_incompressible(
        double* dy_ptr,
        double radius,
        double* y_ptr,
        const void* void_args_ptr,
        PreEvalFunc eos_func) noexcept nogil:

    # Recast the additional arguments
    cdef RadialSolverDiffeqArgStruct* args_ptr = <RadialSolverDiffeqArgStruct*>void_args_ptr

    # Update Equation of State at this radius value
    cdef double[7] eos_array 
    cdef double* eos_array_ptr = &eos_array[0]
    # Call equation of state solution for this layer.
    args_ptr.eos_solution_ptr.call(radius, eos_array_ptr)
    # Pull out results.
    # The EOS stores 7 doubles:
    #   0: Gravity
    #   1: Pressure
    #   2: Density
    #   3: Shear Mod (real)
    #   4: Shear Mod (imag)
    #   5: Bulk Mod (real)
    #   6: Bulk Mod (imag)
    cdef double gravity                  = eos_array_ptr[0]
    cdef double density                  = eos_array_ptr[2]
    cdef double complex shear_modulus    = cf_build_dblcmplx(eos_array_ptr[3], eos_array_ptr[4])
    cdef double complex bulk_modulus     = cf_build_dblcmplx(eos_array_ptr[5], eos_array_ptr[6])

    # Pull out y values
    cdef double complex y1 = cf_build_dblcmplx(y_ptr[0], y_ptr[1])
    cdef double complex y2 = cf_build_dblcmplx(y_ptr[2], y_ptr[3])
    cdef double complex y3 = cf_build_dblcmplx(y_ptr[4], y_ptr[5])
    cdef double complex y4 = cf_build_dblcmplx(y_ptr[6], y_ptr[7])
    cdef double complex y5 = cf_build_dblcmplx(y_ptr[8], y_ptr[9])
    cdef double complex y6 = cf_build_dblcmplx(y_ptr[10], y_ptr[11])

    # Optimizations
    cdef double r_inverse               = 1. / radius
    cdef double density_gravity         = density * gravity
    cdef double grav_term               = args_ptr.grav_coeff * density
    cdef double complex two_shear_r_inv = 2. * shear_modulus * r_inverse
    cdef double complex y1_y3_term      = 2. * y1 - args_ptr.llp1 * y3

    # Solve for radial derivatives
    cdef double complex dy1 = \
        -1. * y1_y3_term * r_inverse

    cdef double complex dy2 = \
        r_inverse * (
            y1 * (12. * shear_modulus * r_inverse - 4. * density_gravity) +
            y3 * args_ptr.llp1 * (density_gravity - 6. * shear_modulus * r_inverse) +
            y4 * args_ptr.llp1 +
            y5 * density * args_ptr.lp1 +
            y6 * -density * radius
        )

    cdef double complex dy3 = \
        y1 * -r_inverse + \
        y3 * r_inverse + \
        y4 * (1. / shear_modulus)

    cdef double complex dy4 = \
        r_inverse * (
            y1 * (density_gravity - 3. * two_shear_r_inv) +
            y2 * -1. +
            y3 * (two_shear_r_inv * (2. * args_ptr.llp1 - 1.)) +
            y4 * -3. +
            y5 * -density
        )

    cdef double complex dy5 = \
        y1 * grav_term + \
        y5 * -args_ptr.lp1 * r_inverse + \
        y6

    cdef double complex dy6 = \
        r_inverse * (
            y1 * grav_term * args_ptr.lm1 +
            y6 * args_ptr.lm1 +
            y1_y3_term * grav_term
        )

    # Convert back to floats
    dy_ptr[0]  = dy1.real
    dy_ptr[1]  = dy1.imag
    dy_ptr[2]  = dy2.real
    dy_ptr[3]  = dy2.imag
    dy_ptr[4]  = dy3.real
    dy_ptr[5]  = dy3.imag
    dy_ptr[6]  = dy4.real
    dy_ptr[7]  = dy4.imag
    dy_ptr[8]  = dy5.real
    dy_ptr[9]  = dy5.imag
    dy_ptr[10] = dy6.real
    dy_ptr[11] = dy6.imag


cdef void cf_liquid_dynamic_compressible(
        double* dy_ptr,
        double radius,
        double* y_ptr,
        const void* void_args_ptr,
        PreEvalFunc eos_func) noexcept nogil:

    # Recast the additional arguments
    cdef RadialSolverDiffeqArgStruct* args_ptr = <RadialSolverDiffeqArgStruct*>void_args_ptr

    # Update Equation of State at this radius value
    cdef double[7] eos_array 
    cdef double* eos_array_ptr = &eos_array[0]
    # Call equation of state solution for this layer.
    args_ptr.eos_solution_ptr.call(radius, eos_array_ptr)
    # Pull out results.
    # The EOS stores 7 doubles:
    #   0: Gravity
    #   1: Pressure
    #   2: Density
    #   3: Shear Mod (real)
    #   4: Shear Mod (imag)
    #   5: Bulk Mod (real)
    #   6: Bulk Mod (imag)
    cdef double gravity                  = eos_array_ptr[0]
    cdef double density                  = eos_array_ptr[2]
    cdef double complex shear_modulus    = cf_build_dblcmplx(eos_array_ptr[3], eos_array_ptr[4])
    cdef double complex bulk_modulus     = cf_build_dblcmplx(eos_array_ptr[5], eos_array_ptr[6])

    # Pull out y values
    # For the dynamic version, y4 = 0 always in a liquid layer and y3 is defined by y1, y2, and y5 analytically
    cdef double complex y1 = cf_build_dblcmplx(y_ptr[0], y_ptr[1])
    cdef double complex y2 = cf_build_dblcmplx(y_ptr[2], y_ptr[3])
    cdef double complex y5 = cf_build_dblcmplx(y_ptr[4], y_ptr[5])
    cdef double complex y6 = cf_build_dblcmplx(y_ptr[6], y_ptr[7])

    # Optimizations
    cdef double r_inverse         = 1. / radius
    cdef double density_gravity   = density * gravity
    cdef double f2                = args_ptr.frequency * args_ptr.frequency
    cdef double dynamic_term_no_r = -f2 * density
    cdef double dynamic_term      = dynamic_term_no_r * radius
    cdef double grav_term         = args_ptr.grav_coeff * density

    # Check if dynamic term is close to zero. It will always be negative so compare to negative eps
    # if dynamic_term < EPS_100:
    #     # TODO: is faking this okay?
    #     dynamic_term = EPS_100

    # For the liquid layer it is assumed that the shear modulus is zero so the lame parameter simply
    #    equals the bulk modulus. Until bulk dissipation is considered, it will always be real-valued
    cdef double complex lame_inverse = 1. / bulk_modulus

    # y3 derivative is undetermined for a liquid layer, but we can calculate its value which is still used in the
    #   other derivatives.
    y3 = (1. / dynamic_term) * (y2 - density_gravity * y1 + density * y5)
    y1_y3_term = 2. * y1 - args_ptr.llp1 * y3

    # Eqs. 11--14 in KMN15 equations look like they don't match TS72 because they applied the rheology already.
    #    and substituted y3.
    # We will use TS72 eq. 87 to allow for a generic rheology and bulk dissipation.
    # dy2 contain all three of: dynamic, viscoelastic, and gravitational terms.
    cdef double complex dy1 = \
        y2 * lame_inverse - \
        y1_y3_term * r_inverse

    # TODO: In the solid version there is a [2. * (lame + shear_modulus) * r_inverse] coefficient for y1_y3_term
    #   In TS72 the first term is gone. Shouldn't Lame + mu = Lame = Bulk for liquid layer?
    cdef double complex dy2 = \
        y1 * (dynamic_term_no_r - 2. * density_gravity * r_inverse) + \
        y5 * density * args_ptr.lp1 * r_inverse - \
        y6 * density - \
        y1_y3_term * density_gravity * r_inverse

    cdef double complex dy5 = \
        y1 * grav_term - \
        y5 * args_ptr.lp1 * r_inverse + \
        y6

    cdef double complex dy6 = \
        r_inverse * (
            args_ptr.lm1 * (y1 * grav_term + y6) +
            y1_y3_term * grav_term
        )

    # Convert back to floats
    dy_ptr[0] = dy1.real
    dy_ptr[1] = dy1.imag
    dy_ptr[2] = dy2.real
    dy_ptr[3] = dy2.imag
    dy_ptr[4] = dy5.real
    dy_ptr[5] = dy5.imag
    dy_ptr[6] = dy6.real
    dy_ptr[7] = dy6.imag


cdef void cf_liquid_dynamic_incompressible(
        double* dy_ptr,
        double radius,
        double* y_ptr,
        const void* void_args_ptr,
        PreEvalFunc eos_func) noexcept nogil:

    # Recast the additional arguments
    cdef RadialSolverDiffeqArgStruct* args_ptr = <RadialSolverDiffeqArgStruct*>void_args_ptr

    # Update Equation of State at this radius value
    cdef double[7] eos_array 
    cdef double* eos_array_ptr = &eos_array[0]
    # Call equation of state solution for this layer.
    args_ptr.eos_solution_ptr.call(radius, eos_array_ptr)
    # Pull out results.
    # The EOS stores 7 doubles:
    #   0: Gravity
    #   1: Pressure
    #   2: Density
    #   3: Shear Mod (real)
    #   4: Shear Mod (imag)
    #   5: Bulk Mod (real)
    #   6: Bulk Mod (imag)
    cdef double gravity                  = eos_array_ptr[0]
    cdef double density                  = eos_array_ptr[2]
    cdef double complex shear_modulus    = cf_build_dblcmplx(eos_array_ptr[3], eos_array_ptr[4])
    cdef double complex bulk_modulus     = cf_build_dblcmplx(eos_array_ptr[5], eos_array_ptr[6])

    # Pull out y values
    # For the dynamic version, y4 = 0 always in a liquid layer and y3 is defined by y1, y2, and y5 analytically
    cdef double complex y1 = cf_build_dblcmplx(y_ptr[0], y_ptr[1])
    cdef double complex y2 = cf_build_dblcmplx(y_ptr[2], y_ptr[3])
    cdef double complex y5 = cf_build_dblcmplx(y_ptr[4], y_ptr[5])
    cdef double complex y6 = cf_build_dblcmplx(y_ptr[6], y_ptr[7])

    # Optimizations
    cdef double r_inverse       = 1. / radius
    cdef double density_gravity = density * gravity
    cdef double dynamic_term    = -args_ptr.frequency * args_ptr.frequency * density * radius
    cdef double grav_term       = args_ptr.grav_coeff * density

    # Check if dynamic term is close to zero. It will always be negative so compare to negative eps
    # if dynamic_term < EPS_100:
    #     # TODO: is faking this okay?
    #     dynamic_term = EPS_100

    # y3 derivative is undetermined for a liquid layer, but we can calculate its value which is still used in the
    #   other derivatives.
    cdef double complex y3 = (1. / dynamic_term) * (y2 + density * y5 - density_gravity * y1)
    cdef double complex y1_y3_term = 2. * y1 - args_ptr.llp1 * y3

    cdef double complex dy1 = \
        y1_y3_term * -r_inverse

    cdef double complex dy2 = \
        r_inverse * (
            y1 * (dynamic_term - 2. * density_gravity) +
            y5 * density * args_ptr.lp1 +
            y6 * -density * radius +
            # TODO: In the solid version there is a [2. * (lame + shear_modulus) * r_inverse] coefficient for y1_y3_term
            #   In TS72 the first term is gone. Shouldn't Lame + mu = Lame = Bulk for liquid layer?
            y1_y3_term * -density_gravity
        )

    cdef double complex dy5 = \
        y1 * grav_term + \
        y5 * -args_ptr.lp1 * r_inverse + \
        y6

    cdef double complex dy6 = \
        r_inverse * (
            y1 * grav_term * args_ptr.lm1 +
            y6 * args_ptr.lm1 +
            y1_y3_term * grav_term
        )

    # Convert back to floats
    dy_ptr[0] = dy1.real
    dy_ptr[1] = dy1.imag
    dy_ptr[2] = dy2.real
    dy_ptr[3] = dy2.imag
    dy_ptr[4] = dy5.real
    dy_ptr[5] = dy5.imag
    dy_ptr[6] = dy6.real
    dy_ptr[7] = dy6.imag


cdef void cf_liquid_static_incompressible(
        double* dy_ptr,
        double radius,
        double* y_ptr,
        const void* void_args_ptr,
        PreEvalFunc eos_func) noexcept nogil:

    # Recast the additional arguments
    cdef RadialSolverDiffeqArgStruct* args_ptr = <RadialSolverDiffeqArgStruct*>void_args_ptr

    # Update Equation of State at this radius value
    cdef double[7] eos_array 
    cdef double* eos_array_ptr = &eos_array[0]
    # Call equation of state solution for this layer.
    args_ptr.eos_solution_ptr.call(radius, eos_array_ptr)
    # Pull out results.
    # The EOS stores 7 doubles:
    #   0: Gravity
    #   1: Pressure
    #   2: Density
    #   3: Shear Mod (real)
    #   4: Shear Mod (imag)
    #   5: Bulk Mod (real)
    #   6: Bulk Mod (imag)
    cdef double gravity                  = eos_array_ptr[0]
    cdef double density                  = eos_array_ptr[2]
    cdef double complex shear_modulus    = cf_build_dblcmplx(eos_array_ptr[3], eos_array_ptr[4])
    cdef double complex bulk_modulus     = cf_build_dblcmplx(eos_array_ptr[5], eos_array_ptr[6])

    # For the static liquid version, only y5 and y7 are defined.
    cdef double complex y5 = cf_build_dblcmplx(y_ptr[0], y_ptr[1])
    cdef double complex y7 = cf_build_dblcmplx(y_ptr[2], y_ptr[3])

    # Optimizations
    cdef double r_inverse = 1. / radius
    cdef double grav_term = args_ptr.grav_coeff * density / gravity

    # See Eq. 18 in S75
    cdef double complex dy5 = \
        y5 * (grav_term - args_ptr.lp1 * r_inverse) + \
        y7

    cdef double complex dy7 = \
        y5 * 2. * args_ptr.lm1 * r_inverse * grav_term + \
        y7 * (args_ptr.lm1 * r_inverse - grav_term)

    # Convert back to floats; stor in output array
    dy_ptr[0] = dy5.real
    dy_ptr[1] = dy5.imag
    dy_ptr[2] = dy7.real
    dy_ptr[3] = dy7.imag


cdef DiffeqFuncType cf_find_layer_diffeq(
        int layer_type,
        int layer_is_static,
        int layer_is_incomp) noexcept nogil:
    
    if layer_type == 0:
        # Solid
        if layer_is_static == 1:
            # Solid-Static
            if layer_is_incomp == 1:
                # Solid-Static-Incomp
                return cf_solid_static_incompressible
            else:
                # Solid-Static-Comp
                return cf_solid_static_compressible
        else:
            # Solid-Dynamic
            if layer_is_incomp:
                # Solid-Dynamic-Incomp
                return cf_solid_dynamic_incompressible
                
            else:
                # Solid-Dynamic-Comp
                return cf_solid_dynamic_compressible
    else:
        # Liquid
        if layer_is_static == 1:
            # Liquid-Static
            if layer_is_incomp == 1:
                # Liquid-Static-Incomp
                return cf_liquid_static_incompressible
            else:
                # Liquid-Static-Comp
                # TODO: Compressible is the same as the incompressible version; Check if this is true.
                return cf_liquid_static_incompressible
        else:
            # Liquid-Dynamic
            if layer_is_incomp:
                # Liquid-Dynamic-Incomp
                return cf_liquid_dynamic_incompressible
                
            else:
                # Liquid-Dynamic-Comp
                return cf_liquid_dynamic_compressible

cdef struct RadialSolverDiffeqArgStruct:
    double degree_l
    double lp1
    double lm1
    double llp1
    double G
    double grav_coeff
    double frequency
    CySolverResult* eos_solution_ptr


cdef void cf_apply_surface_bc(
        double complex* constant_vector_ptr,
        int* bc_solution_info_ptr,
        double* bc_pointer,
        double complex* uppermost_y_per_solution_ptr,
        double surface_gravity,
        double G_to_use,
        unsigned char num_sols,
        unsigned char max_num_y,
        unsigned char ytype_i,
        int layer_type,
        bint layer_is_static,
        bint layer_is_incomp
        ) noexcept nogil:
    
    # Variables used to solve the linear equation at the planet's surface.
    # These are passed to the LAPACK solver.
    # NRHS = number of solutions that will be solved at the same time. Only one will be solved per radial_solver call.
    cdef int lapack_nrhs = 1
    cdef int* lapack_nrhs_ptr = &lapack_nrhs

    # IPIV = Integer pivot array that is an additional output provided by ZGESV. It is not used but must be provided.
    #  It must be at least as large as the largest dimension of the input matrix, for this work that is 3.
    cdef int[10] lapack_ipiv
    cdef int* lapack_ipiv_ptr = &lapack_ipiv[0]

    cdef int num_sols_int = <int>num_sols
    cdef int* num_sols_int_ptr = &num_sols_int

    # Allocate surface matrices. We only need one of these (which one depends on the uppermost layer type).
    # But, since there are only 3 and all of them are small, we will just allocate all of them separately on the stack.
    cdef double complex[3][3] surface_matrix_solid
    cdef double complex[2][2] surface_matrix_liquid_dynamic
    cdef double complex[1][1] surface_matrix_liquid_static
    cdef double complex* surface_matrix_ptr

    # Create coefficient matrix based on surface layer type.
    if (layer_type == 0):
        # Solid layer
        # Set pointer to correct matrix
        surface_matrix_ptr = &surface_matrix_solid[0][0]

        # At the surface: y_2 = S_1; y_4 = S_4; y_6 = S_6 [See: B.37 in KTC21; 16 in KMN15]
        # We will set the constant vector equal to the surface boundary condition.
        #  It will be overwritten with the solution to the linear solution.
        constant_vector_ptr[0] = bc_pointer[ytype_i * 3 + 0]
        constant_vector_ptr[1] = bc_pointer[ytype_i * 3 + 1]
        constant_vector_ptr[2] = bc_pointer[ytype_i * 3 + 2]

        # The definitions above need to be transposed as the LAPACK solver TidalPy uses requires
        #  FORTRAN-ordered arrays.
        surface_matrix_ptr[0] = uppermost_y_per_solution_ptr[0 * max_num_y + 1]
        surface_matrix_ptr[1] = uppermost_y_per_solution_ptr[0 * max_num_y + 3]
        surface_matrix_ptr[2] = uppermost_y_per_solution_ptr[0 * max_num_y + 5]
        surface_matrix_ptr[3] = uppermost_y_per_solution_ptr[1 * max_num_y + 1]
        surface_matrix_ptr[4] = uppermost_y_per_solution_ptr[1 * max_num_y + 3]
        surface_matrix_ptr[5] = uppermost_y_per_solution_ptr[1 * max_num_y + 5]
        surface_matrix_ptr[6] = uppermost_y_per_solution_ptr[2 * max_num_y + 1]
        surface_matrix_ptr[7] = uppermost_y_per_solution_ptr[2 * max_num_y + 3]
        surface_matrix_ptr[8] = uppermost_y_per_solution_ptr[2 * max_num_y + 5]

    else:
        if layer_is_static:
            # Set pointer to correct matrix
            surface_matrix_ptr = &surface_matrix_liquid_dynamic[0][0]

            # Unlike the dynamic liquid layer, a static liquid layer's y_2 is undefined. That leads to one less boundary condition
            #  and one less solution (1 total).
            #  At the surface, y_7 = S_7 [See: Eq. 17, 10 in S74]

            # We will set the constant vector equal to the surface boundary condition.
            #  It will be overwritten with the solution to the linear solution.
            # y_7 = y_6 + (4 pi G / g) y_2
            constant_vector_ptr[0] = \
                bc_pointer[ytype_i * 3 + 2] + \
                bc_pointer[ytype_i * 3 + 0] * (4. * pi * G_to_use / surface_gravity)

            # These are unused. Set to NAN so if they do get used we might be able to catch it.
            constant_vector_ptr[1] = NAN
            constant_vector_ptr[2] = NAN

            # Note: for a static liquid layer, y_7 held in index 1 (index 0 is y_5).
            surface_matrix_ptr[0] = uppermost_y_per_solution_ptr[0 * max_num_y + 1]
        else:
            # Set pointer to correct matrix
            surface_matrix_ptr = &surface_matrix_liquid_static[0][0]

            # Unlike the solid layer, a liquid layer's y_4 is undefined. That leads to one less boundary condition and one
            #  less solution (2 total).
            #  At the surface, y_2 = S_1; y_6 = S_6 [See: Eq. B.38 in KTC21; Eq. 17 in KMN15

            # We will set the constant vector equal to the surface boundary condition.
            #  It will be overwritten with the solution to the linear solution.
            # The surface boundary condition will still have 3 members. Drop the one related to y_4
            constant_vector_ptr[0] = bc_pointer[ytype_i * 3 + 0]
            constant_vector_ptr[1] = bc_pointer[ytype_i * 3 + 2]

            # The last constant is unused. Set to NAN so if they do get used we might be able to catch it.
            constant_vector_ptr[2] = NAN

            # Build y-solution matrix to be applied to the surface.
            # Note: for a dynamic liquid, y_2 and y_6 are held at indices 1 and 3 respectively
            surface_matrix_ptr[0] = uppermost_y_per_solution_ptr[0 * max_num_y + 1]
            surface_matrix_ptr[1] = uppermost_y_per_solution_ptr[0 * max_num_y + 3]
            surface_matrix_ptr[2] = uppermost_y_per_solution_ptr[1 * max_num_y + 1]
            surface_matrix_ptr[3] = uppermost_y_per_solution_ptr[1 * max_num_y + 3]

    # Find the solution to the linear equation at the surface surface_matrix @ constant_vector = bc
    # ZGESV computes the solution to system of linear equations A * X = B for GE matrices
    # See https://www.math.utah.edu/software/lapack/lapack-z/zgesv.html
    zgesv(
        num_sols_int_ptr,     # (Input)
        lapack_nrhs_ptr,      # (Input)
        surface_matrix_ptr,   # A; (Input & Output)
        num_sols_int_ptr,     # (Input)
        lapack_ipiv_ptr,      # (Output)
        constant_vector_ptr,  # B -> X (Input & Output)
        num_sols_int_ptr,     # (Input)
        bc_solution_info_ptr  # (Ouput)
        )


cdef void cf_get_surface_bc(
    double* boundary_conditions_ptr,
    int* bc_model_ptr,
    size_t num_bcs,
    double radius_to_use,
    double bulk_density_to_use,
    double degree_l_dbl,
    ) noexcept nogil:
    """Find the surface boundary condition. """

    # `num_bcs` should equal the length of `bc_model_ptr`
    if num_bcs > 5:
        printf(
            "Unsupported number of boundaries conditions encountered."
            " Provided: %d when maximum supported is 5.", num_bcs)
        exit(-1)
    elif num_bcs <= 0:
        printf(
            "Unsupported number of boundaries conditions encountered."
            " Provided: %d when minimum supported is 1.", num_bcs)
        exit(-1)
    cdef size_t i, j

    # Inititalize all boundary conditions to NaN
    # 15 = 5 (max_num_solutions) * 3 (number of surface conditions)
    for i in range(15):
        boundary_conditions_ptr[i] = NAN
    
    for j in range(num_bcs):
        if bc_model_ptr[j] == 0:
            # Free Surface
            boundary_conditions_ptr[j * 3 + 0] = 0.
            boundary_conditions_ptr[j * 3 + 1] = 0.
            boundary_conditions_ptr[j * 3 + 2] = 0.
        elif bc_model_ptr[j] == 1:
            # Tidal Potential
            boundary_conditions_ptr[j * 3 + 0] = 0.
            boundary_conditions_ptr[j * 3 + 1] = 0.
            boundary_conditions_ptr[j * 3 + 2] = (2. * degree_l_dbl + 1.) / radius_to_use
        elif bc_model_ptr[j] == 2:
            # Loading Potential
            boundary_conditions_ptr[j * 3 + 0] = (-1. / 3.) * (2. * degree_l_dbl + 1.) * bulk_density_to_use
            boundary_conditions_ptr[j * 3 + 1] = 0.
            boundary_conditions_ptr[j * 3 + 2] = (2. * degree_l_dbl + 1.) / radius_to_use
        else:
            printf(
                "Unknown boundary condition model: %d. Supported models are:\n"
                " 0: Free Surface.\n"
                " 1: Tidal Potential.\n"
                " 2: Loading Potential.\n", bc_model_ptr[j])
            exit(-1)



cdef void cf_collapse_layer_solution(
        double complex* solution_ptr,
        double complex* constant_vector_ptr,
        double complex** storage_by_solution,
        double* layer_radius_ptr,
        double* layer_density_ptr,
        double* layer_gravity_ptr,
        double frequency_to_use,
        size_t layer_start_index,
        size_t num_layer_slices,
        unsigned char num_sols,
        unsigned char max_num_y,
        unsigned char num_ys,
        unsigned char num_output_ys,
        unsigned char ytype_i,
        int layer_type,
        bint layer_is_static,
        bint layer_is_incomp
        ) noexcept nogil:

    # Use constant vectors to find the full y from all of the solutions in this layer
    cdef unsigned char solution_i
    cdef unsigned char y_i
    cdef unsigned char y_rhs_i
    cdef unsigned char lhs_y_index
    cdef size_t slice_i
    cdef size_t slice_i_shifted
    cdef size_t slice_end
    cdef bint calculate_y3

    slice_end = layer_start_index + num_layer_slices
    calculate_y3 = False

    if (not (layer_type == 0)) and (not layer_is_static):
        # For this layer type we can also calculate y3 based on the other ys. We will do this at the end.
        calculate_y3 = True
    
    y_rhs_i = 0
    y_i = 0
    while max_num_y > y_i:
        lhs_y_index = ytype_i * max_num_y + y_i
        # Bail out early for ys that the layer type is not defined for.
        if (layer_type == 0):
            # Solid layers have the same number of ys as the max_y_num.
            y_rhs_i = y_i
        else:
            # Liquid layers have less ys than max_y_num. Set y_rhs_i to the appropriate index.
            if layer_is_static:
                # Liquid static layers only has y5 (stored at index 0).
                if y_i == 4:
                    y_rhs_i = 0
                else:
                    # All results should be NAN which the solution pointer should already be set to. Continue.
                    y_i += 1
                    continue
            else:
                # Liquid dynamic layers have y1, y2, y5, y6 (indices 0, 1, 2, 3)
                # For this layer type we can also calculate y3 based on the other ys.
                if y_i < 2:
                    y_rhs_i = y_i
                elif (y_i > 3) and (y_i < 6):
                    y_rhs_i = y_i - 2
                else:
                    # y4 == NAN for liquid dynamic layers; y3 will be calculated later.
                    y_i += 1
                    continue
        
        # Otherwise we will collapse the solutions together for each slice.
        solution_i = 0
        while num_sols > solution_i:
            # New solution; reset slice index
            slice_i = 0
            slice_i_shifted = layer_start_index
            while slice_end > slice_i_shifted:
                if solution_i == 0:
                    # Initialize values
                    solution_ptr[slice_i_shifted * num_output_ys + lhs_y_index] = \
                        (constant_vector_ptr[solution_i] *
                        storage_by_solution[solution_i][slice_i * num_ys + y_rhs_i])
                else:
                    # Add new results to old value
                    solution_ptr[slice_i_shifted * num_output_ys + lhs_y_index] += \
                        (constant_vector_ptr[solution_i] *
                        storage_by_solution[solution_i][slice_i * num_ys + y_rhs_i])

                slice_i += 1
                slice_i_shifted += 1
            solution_i += 1
        y_i += 1
    
    if calculate_y3:
        # Handle y3 for dynamic liquid layers.
        lhs_y_index = ytype_i * max_num_y
        slice_i = 0
        slice_i_shifted = layer_start_index
        while slice_end > slice_i_shifted:
            solution_ptr[slice_i_shifted * num_output_ys + (lhs_y_index + 2)] = \
                (1. / (frequency_to_use**2 * layer_radius_ptr[slice_i])) * \
                (solution_ptr[slice_i_shifted * num_output_ys + (lhs_y_index + 0)] * layer_gravity_ptr[slice_i] -
                solution_ptr[slice_i_shifted * num_output_ys + (lhs_y_index + 1)] / layer_density_ptr[slice_i] -
                solution_ptr[slice_i_shifted * num_output_ys + (lhs_y_index + 4)])

            slice_i += 1
            slice_i_shifted += 1



cdef void cf_shooting_solver(
        RadialSolutionStorageCC* solution_storage_ptr,
        size_t total_slices,
        double* radius_array_ptr,
        double frequency,
        double planet_bulk_density,
        CySolverResult** eos_solution_bylayer_ptr,
        size_t num_layers,
        int* layer_types_ptr,
        int* is_static_by_layer_ptr,
        int* is_incompressible_by_layer_ptr,
        double* upper_radius_by_layer_ptr,
        size_t* first_slice_index_by_layer_ptr,
        size_t* num_slices_by_layer_ptr,
        size_t num_bc_models,
        int* bc_models_ptr,
        double G_to_use = G,
        unsigned int degree_l = 2,
        cpp_bool use_kamata = False,
        unsigned char integration_method = 1,
        double integration_rtol = 1.0e-6,
        double integration_atol = 1.0e-12,
        cpp_bool scale_rtols_by_layer_type = False,
        size_t max_num_steps = 500_000,
        size_t expected_size = 500,
        size_t max_ram_MB = 500,
        double max_step = 0,
        cpp_bool verbose = False,
        cpp_bool raise_on_fail = False
        ) noexcept nogil:
    """ Solves the viscoelastic-gravitational problem for planets using a shooting method.
    """
    # Feedback
    cdef char[256] message
    cdef char* message_ptr = &message[0]
    cdef cpp_bool error = False
    strcpy(message_ptr, 'RadialSolver.ShootingMethod:: Starting integration\n')
    solution_storage_ptr.set_message(message_ptr)
    if verbose:
        printf(message_ptr)

    # General indexing
    cdef double radius_check = NAN
    cdef size_t i
    cdef size_t current_layer_i
    cdef size_t first_slice_index
    cdef size_t slice_i
    # Indexing for: Solution | ys | ytypes
    cdef unsigned char solution_i
    cdef unsigned char y_i
    cdef unsigned char ytype_i

    # Type conversions
    cdef double degree_l_dbl = <double>degree_l

    # Alias pointers to EOS properties
    cdef double* gravity_array_ptr = solution_storage_ptr.gravity_ptr
    # cdef double* pressure_array_ptr = solution_storage_ptr.pressure_ptr   # Unused
    cdef double* density_array_ptr = solution_storage_ptr.density_ptr

    # Need to recast the storage's shear/bulk double arrays to double complex for local use
    cdef double complex* complex_shear_array_ptr = <double complex*>solution_storage_ptr.shear_mod_ptr
    cdef double complex* complex_bulk_array_ptr  = <double complex*>solution_storage_ptr.bulk_mod_ptr

    # Pull out key information
    cdef size_t top_slice_i = total_slices - 1
    
    # Pull out any constants now that arrays have had dimensional protocol applied to them.
    cdef double planet_radius   = radius_array_ptr[top_slice_i]
    cdef double surface_gravity = gravity_array_ptr[top_slice_i]

    # Find boundary condition at the top of the planet -- this is dependent on the forcing type.
    #     Tides (default here) follow the (y2, y4, y6) = (0, 0, (2l+1)/R) rule
    # The [5] represents the maximum number of solvers that can be invoked with a single call to radial_solver
    cdef size_t num_ytypes = num_bc_models

    # 15 = 5 (max_num_solutions) * 3 (number of surface conditions)
    cdef double[15] boundary_conditions
    cdef double* bc_pointer = &boundary_conditions[0]
    cf_get_surface_bc(
        bc_pointer,  # Changed parameter
        bc_models_ptr,
        num_ytypes,
        planet_radius,
        planet_bulk_density,
        degree_l_dbl
        )

    # Integration information
    # Max step size
    cdef double max_step_to_use        = NAN
    cdef cpp_bool max_step_from_arrays = False
    if max_step == 0:
        # If max_step is zero use the array information to determine max_step_size
        max_step_from_arrays = True
    else:
        # Otherwise use user input.
        max_step_to_use = max_step

    # Setup tolerance arrays
    # For simplicity just make these all as large as the maximum number of ys.
    # Maximum number of ys = 6. Then 2x for conversion from complex to real
    cdef double[12] rtols_array
    cdef double[12] atols_array
    cdef double* rtols_ptr = &rtols_array[0]
    cdef double* atols_ptr = &atols_array[0]

    for i in range(12):
        rtols_ptr[i] = NAN
        atols_ptr[i] = NAN

    # Create storage for flags and information about each layer.
    cdef vector[size_t] num_solutions_by_layer_vec = vector[size_t]()
    num_solutions_by_layer_vec.reserve(num_layers)
    cdef size_t* num_solutions_by_layer_ptr = &num_solutions_by_layer_vec[0]

    # Opt: The bools above could be stores in a single char variable (per layer).
    #  Eg., 0x00 All false, 0x01 is solid, 0x10 is static and liquid, 0x11 is static and solid, etc.

    cdef int layer_type
    cdef int layer_is_static
    cdef int layer_is_incomp
    cdef int layer_below_type
    cdef int layer_below_is_static
    cdef int layer_below_is_incomp
    cdef size_t layer_slices
    cdef unsigned char num_sols
    cdef unsigned char num_ys
    cdef unsigned char num_ys_dbl
    cdef unsigned char layer_below_num_sols
    cdef unsigned char layer_below_num_ys
    cdef double layer_upper_radius = NAN
    cdef double layer_rtol_real    = NAN
    cdef double layer_rtol_imag    = NAN
    cdef double layer_atol_real    = NAN
    cdef double layer_atol_imag    = NAN

    for current_layer_i in range(num_layers):
        # Pull out information on this layer
        layer_type         = layer_types_ptr[current_layer_i]
        layer_is_static    = is_static_by_layer_ptr[current_layer_i]
        layer_is_incomp    = is_incompressible_by_layer_ptr[current_layer_i]
        layer_upper_radius = upper_radius_by_layer_ptr[current_layer_i]

        # Find number of solutions based on this layer's assumptions
        num_sols = cf_find_num_shooting_solutions(
            layer_type,
            layer_is_static,
            layer_is_incomp
            )
        num_ys = 2 * num_sols
        num_solutions_by_layer_ptr[current_layer_i] = num_sols

    # We have all the size information needed to build storage pointers
    # Main storage pointer is setup like [current_layer_i][solution_i][y_i + r_i]
    cdef double complex*** main_storage_ptr = <double complex ***> allocate_mem(
        num_layers * sizeof(double complex**),
        'main_storage_ptr (radial_solver; init)'
        )

    cdef double complex** storage_by_solution_ptr = NULL
    cdef double complex* storage_by_y_ptr = NULL

    for current_layer_i in range(num_layers):
        num_sols     = num_solutions_by_layer_ptr[current_layer_i]
        layer_slices = num_slices_by_layer_ptr[current_layer_i]
        # Number of ys = 2x num sols
        num_ys = 2 * num_sols

        storage_by_solution_ptr = <double complex **> allocate_mem(
            num_sols * sizeof(double complex*), 
            'storage_by_solution_ptr (radial_solver; init)'
            ) 

        for solution_i in range(num_sols):
            storage_by_y_ptr = <double complex *> allocate_mem(
                layer_slices * num_ys * sizeof(double complex),
                'storage_by_y_ptr (radial_solver; init)'
                )

            storage_by_solution_ptr[solution_i] = storage_by_y_ptr
            storage_by_y_ptr = NULL
        main_storage_ptr[current_layer_i] = storage_by_solution_ptr
        storage_by_solution_ptr = NULL

    # Create storage for uppermost ys for each solution. We don't know how many solutions or ys per layer so assume the
    #  worst.
    cdef double complex[18] uppermost_y_per_solution
    cdef double complex* uppermost_y_per_solution_ptr = &uppermost_y_per_solution[0]

    for i in range(18):
        uppermost_y_per_solution_ptr[i] = cmplx_NAN

    # Layer specific pointers; set the size based on the layer with the most slices.
    cdef double* layer_radius_ptr
    cdef double* layer_density_ptr
    cdef double* layer_gravity_ptr
    cdef double complex* layer_bulk_mod_ptr
    cdef double complex* layer_shear_mod_ptr

    # Properties at top and bottom of layer
    cdef double[2] radial_span
    cdef double* radial_span_ptr = &radial_span[0]
    radial_span_ptr[0] = NAN
    radial_span_ptr[1] = NAN

    cdef double radius_lower  = NAN
    cdef double radius_upper  = NAN
    cdef double density_lower = NAN
    cdef double density_upper = NAN
    cdef double gravity_lower = NAN
    cdef double gravity_upper = NAN
    cdef double complex bulk_lower  = cmplx_NAN
    cdef double complex bulk_upper  = cmplx_NAN
    cdef double complex shear_upper = cmplx_NAN
    cdef double complex shear_lower = cmplx_NAN

    # Properties at interfaces between layers
    cdef double static_liquid_density = NAN
    cdef double interface_gravity = NAN
    cdef double last_layer_upper_gravity = NAN
    cdef double last_layer_upper_density = NAN

    # Starting solutions (initial conditions / lower boundary conditions)
    # Allocate memory for the initial value arrays now. We don't know the number of solutions or ys. But the max number
    #  is not all that different from the min. So it is more efficient to assume the largest size.
    # Largest size = 6 (ys) x 3 (sols) = 18
    # For the "only_real" this is further multiplied by 2 since we convert the number of _complex_ ys to 2x _real_ ys
    cdef double complex[18] initial_y
    cdef double complex* initial_y_ptr = &initial_y[0]
    cdef double[36] initial_y_only_real
    cdef double* initial_y_only_real_ptr = &initial_y_only_real[0]
    cdef cpp_bool starting_y_check = False

    # Intermediate values
    cdef double complex dcomplex_tmp = cmplx_NAN

    # No matter the results of the integration, we know the shape and size of the final solution.
    # The number of rows will depend on if the user wants to simultaneously calculate loading Love numbers.
    cdef size_t num_output_ys = MAX_NUM_Y * num_ytypes

    # Get a reference pointer to solution array
    cdef double* solution_dbl_ptr = solution_storage_ptr.full_solution_ptr
    # Cast the solution pointer from double to double complex
    cdef double complex* solution_ptr = <double complex*>solution_dbl_ptr

    # During collapse, variables for the layer above the target one are used. Declare these and preset them.
    cdef double layer_above_lower_gravity
    cdef double layer_above_lower_density
    cdef double liquid_density_at_interface
    cdef int layer_above_type
    cdef cpp_bool layer_above_is_static
    cdef cpp_bool layer_above_is_incomp

    # Each layer will have an equation of state solution which will be called during integration to ensure 
    # an accurate interpolation occurs at each radius value
    cdef CySolverResult* eos_solution_ptr = NULL

    # Layer's differential equation will vary by layer type
    cdef CySolveOutput integration_solution
    cdef CySolverResult* integration_solution_ptr
    cdef double* integrator_data_ptr = NULL
    cdef RadialSolverDiffeqArgStruct diffeq_args
    cdef RadialSolverDiffeqArgStruct* diffeq_args_ptr = &diffeq_args
    cdef void* diffeq_args_void_ptr                   = <void*>diffeq_args_ptr
    cdef PreEvalFunc diffeq_preeval_ptr               = NULL
    cdef DiffeqFuncType layer_diffeq                  = NULL

    # Set diffeq inputs that do not change with layer
    diffeq_args_ptr.degree_l   = degree_l_dbl
    diffeq_args_ptr.lp1        = degree_l_dbl + 1.0
    diffeq_args_ptr.lm1        = degree_l_dbl - 1.0
    diffeq_args_ptr.llp1       = degree_l_dbl * (degree_l_dbl + 1.0)
    diffeq_args_ptr.G          = G_to_use
    diffeq_args_ptr.grav_coeff = 4.0 * pi * G_to_use
    diffeq_args_ptr.frequency  = frequency

    # The constant vectors are the same size as the number of solutions in the layer. But since the largest they can
    #  ever be is 3, it is more efficient to just preallocate them on the stack rather than dynamically allocate them
    #  on the heap.
    cdef double complex[3] constant_vector
    cdef double complex* constant_vector_ptr = &constant_vector[0]
    cdef double complex[3] layer_above_constant_vector
    cdef double complex* layer_above_constant_vector_ptr = &layer_above_constant_vector[0]
    cdef double complex[6] surface_solutions
    cdef double complex* surface_solutions_ptr = &surface_solutions[0]

    for i in range(6):
        if i < 3:
            constant_vector_ptr[i] = cmplx_NAN
            layer_above_constant_vector_ptr[i] = cmplx_NAN
        surface_solutions_ptr[i] = cmplx_NAN

    # Variables used to solve the linear equation at the planet's surface.
    # Info = flag set by the solver. Set equal to -999. This will indicate that the solver has not been called yet.
    cdef int bc_solution_info = -999

    # Shifted or reversed indices used during collapse.
    cdef size_t layer_i_reversed

    while solution_storage_ptr.success:
        for current_layer_i in range(num_layers):
            # Get layer's index information
            layer_slices = num_slices_by_layer_ptr[current_layer_i]
            first_slice_index = first_slice_index_by_layer_ptr[current_layer_i]

            # Get layer EOS solution
            # Check if we are using the correct EOS solution (changes with layers)
            eos_solution_ptr = eos_solution_bylayer_ptr[current_layer_i]

            # Get solution and y information
            num_sols   = num_solutions_by_layer_ptr[current_layer_i]
            num_ys     = 2 * num_sols
            num_ys_dbl = 2 * num_ys

            # Setup pointer array slices starting at this layer's beginning
            layer_radius_ptr    = &radius_array_ptr[first_slice_index]
            layer_density_ptr   = &density_array_ptr[first_slice_index]
            layer_gravity_ptr   = &gravity_array_ptr[first_slice_index]
            layer_shear_mod_ptr = &complex_shear_array_ptr[first_slice_index]
            layer_bulk_mod_ptr  = &complex_bulk_array_ptr[first_slice_index]

            # Get physical parameters at the top and bottom of the layer
            radius_lower  = layer_radius_ptr[0]
            gravity_lower = layer_gravity_ptr[0]
            density_lower = layer_density_ptr[0]
            shear_lower   = layer_shear_mod_ptr[0]
            bulk_lower    = layer_bulk_mod_ptr[0]

            radius_upper  = layer_radius_ptr[layer_slices - 1]
            gravity_upper = layer_gravity_ptr[layer_slices - 1]
            density_upper = layer_density_ptr[layer_slices - 1]
            shear_upper   = layer_shear_mod_ptr[layer_slices - 1]
            bulk_upper    = layer_bulk_mod_ptr[layer_slices - 1]
            
            radial_span_ptr[0] = radius_lower
            radial_span_ptr[1] = radius_upper

            # Determine max step size (if not provided by user)
            if max_step_from_arrays:
                # Maximum step size during integration can not exceed the average radial slice size.
                max_step_to_use = (radius_upper - radius_lower) / <double>layer_slices

            # Get assumptions for layer
            layer_type      = layer_types_ptr[current_layer_i]
            layer_is_static = is_static_by_layer_ptr[current_layer_i]
            layer_is_incomp = is_incompressible_by_layer_ptr[current_layer_i]

            # Determine rtols and atols for this layer.
            # Scale rtols by layer type
            for y_i in range(num_ys):
                # Default is that each layer's rtol and atol equal user input.
                # TODO: Change up the tolerance scaling between real and imaginary?
                layer_rtol_real = integration_rtol
                layer_rtol_imag = integration_rtol
                layer_atol_real = integration_atol
                layer_atol_imag = integration_atol

                if scale_rtols_by_layer_type:
                    # Certain layer assumptions can affect solution stability so use additional scales on the relevant rtols
                    # TODO test that these scales are the best. See Issue #44
                    if layer_type == 0:
                        # Solid layer
                        # Scale y2 and y3 by 0.1
                        if (y_i == 1) or (y_i == 2):
                            # Scale both the real and imaginary portions by the same amount.
                            layer_rtol_real *= 0.1
                            layer_rtol_imag *= 0.1
                    else:
                        # Liquid layer
                        if not layer_is_static:
                            # Scale dynamic liquid layer's y2 by additional 0.01.
                            if (y_i == 1):
                                # Scale both the real and imaginary portions by the same amount.
                                layer_rtol_real *= 0.01
                                layer_rtol_imag *= 0.01
                # Populate rtol and atol pointers.
                rtols_ptr[2 * y_i]     = layer_rtol_real
                rtols_ptr[2 * y_i + 1] = layer_rtol_imag
                atols_ptr[2 * y_i]     = layer_atol_real
                atols_ptr[2 * y_i + 1] = layer_atol_imag

            # Set initial y to nan for now.
            # OPT: this is for debugging purposes. could likely be commented out in the future for small performance gain.
            for y_i in range(36):
                if y_i < 18:
                    initial_y_ptr[y_i] = cmplx_NAN
                initial_y_only_real_ptr[y_i] = NAN
            
            if current_layer_i == 0:
                # Use initial condition function
                cf_find_starting_conditions(
                    &solution_storage_ptr.success,
                    solution_storage_ptr.message_ptr,
                    layer_type,
                    layer_is_static,
                    layer_is_incomp,
                    use_kamata,
                    frequency,
                    radius_lower,
                    density_lower,
                    bulk_lower,
                    shear_lower,
                    degree_l,
                    G_to_use,
                    MAX_NUM_Y, 
                    initial_y_ptr,  # Modified Variable
                    starting_y_check
                    )
                if not solution_storage_ptr.success:
                    error = True
                    break
            else:
                layer_below_type      = layer_types_ptr[current_layer_i - 1]
                layer_below_is_static = is_static_by_layer_ptr[current_layer_i - 1]
                layer_below_is_incomp = is_incompressible_by_layer_ptr[current_layer_i - 1]

                # Find gravity at the base interface using bottom of this layer and top of previous.
                interface_gravity = 0.5 * (gravity_lower + last_layer_upper_gravity)

                # Find the density needed for some initial conditions.
                if (layer_type == 0) and (layer_below_type == 0):
                    # Both layers are solid. A liquid interface density is not needed.
                    static_liquid_density = NAN
                elif not (layer_type == 0) and (layer_below_type == 0):
                    # Layer below is solid, this layer is liquid. Use its density.
                    static_liquid_density = density_lower
                elif (layer_type == 0) and not (layer_below_type == 0):
                    # Layer below is liquid, this layer is solid. Use layer below's density.
                    static_liquid_density = last_layer_upper_density
                else:
                    # Both layers are liquid. Choose the layer's density which is static.
                    if layer_is_static and layer_below_is_static:
                        # Both layers are static.
                        # TODO: Not sure what to do here so just use this layer's density.
                        static_liquid_density = density_lower
                    elif layer_is_static and not layer_below_is_static:
                        # This layer is static, one below is not. Use this layer's density
                        static_liquid_density = density_lower
                    elif not layer_is_static and layer_below_is_static:
                        # This layer is dynamic, layer below is static. Use layer below's density
                        static_liquid_density = last_layer_upper_density
                    else:
                        # Both layers are dynamic. Static liquid density is not needed.
                        static_liquid_density = NAN

                # Find the starting values for this layer using the results a the top of the previous layer + an interface
                #  function.
                cf_solve_upper_y_at_interface(
                    uppermost_y_per_solution_ptr,
                    initial_y_ptr,
                    layer_below_num_sols,
                    num_sols,
                    MAX_NUM_Y,
                    layer_below_type,
                    layer_below_is_static,
                    layer_below_is_incomp,
                    layer_type,
                    layer_is_static,
                    layer_is_incomp,
                    interface_gravity,
                    static_liquid_density,
                    G_to_use
                    )

            # Reset the uppermost y value array
            for i in range(18):
                uppermost_y_per_solution_ptr[i] = cmplx_NAN

            # Change initial conditions into 2x real values instead of complex for integration
            for solution_i in range(num_sols):
                for y_i in range(num_ys):
                    dcomplex_tmp = initial_y_ptr[solution_i * MAX_NUM_Y + y_i]
                    initial_y_only_real_ptr[solution_i * MAX_NUM_Y_REAL + 2 * y_i]     = dcomplex_tmp.real
                    initial_y_only_real_ptr[solution_i * MAX_NUM_Y_REAL + 2 * y_i + 1] = dcomplex_tmp.imag

            # Find correct diffeq
            layer_diffeq = cf_find_layer_diffeq(layer_type, layer_is_static, layer_is_incomp)

            # Det diffeq additional arg input's eos solution pointer
            diffeq_args_ptr.eos_solution_ptr = eos_solution_ptr

            # Get storage pointer for this layer
            storage_by_solution_ptr = main_storage_ptr[current_layer_i]

            # Solve for each solution
            for solution_i in range(num_sols):
                ###### Integrate! #######
                integration_solution = cysolve_ivp(
                    layer_diffeq,                                           # Differential equation [DiffeqFuncType]
                    radial_span_ptr,                                        # Radial span [const double*]
                    &initial_y_only_real_ptr[solution_i * MAX_NUM_Y_REAL],  # y0 array [const double*]
                    MAX_NUM_Y_REAL,                                         # Number of ys [const unsigned int]
                    integration_method,                                     # Integration method [unsigned int]
                    NAN,                                                    # Relative Tolerance (as scalar) [double]
                    NAN,                                                    # Absolute Tolerance (as scalar) [double]
                    diffeq_args_void_ptr,                                   # Extra input args to diffeq [void*]
                    0,                                                      # Number of extra outputs tracked [unsigned int]
                    max_num_steps,                                          # Max number of steps (0 = find good value) [size_t]
                    max_ram_MB,                                             # Max amount of RAM allowed [size_t]
                    False,                                                  # Use dense output [bint]
                    layer_radius_ptr,                                       # Interpolate at radius array [double*]
                    layer_slices,                                           # Size of interpolation array [size_t]
                    diffeq_preeval_ptr,                                     # Pre-eval function used in diffeq [PreEvalFunc]
                    rtols_ptr,                                              # Relative Tolerance (as array) [double*]
                    atols_ptr,                                              # Absolute Tolerance (as array) [double*]
                    max_step,                                               # Maximum step size [double]
                    0.0,                                                    # Initial step size (0 = find good value) [doub;e]
                    expected_size                                           # Expected final integration size (0 = find good value) [size_t]
                    )                   
                integration_solution_ptr = integration_solution.get()
                #########################

                # Check for problems
                if not integration_solution_ptr.success:
                    # Problem with integration.
                    error = True
                    sprintf(message_ptr, 'RadialSolver.ShootingMethod:: Integration problem at layer %d; solution %d:\n\t%s.\n', current_layer_i, solution_i, integration_solution_ptr.message_ptr)
                    solution_storage_ptr.set_message(message_ptr)
                    if verbose or raise_on_fail:
                        printf(message_ptr)
                    if raise_on_fail:
                        exit(EXIT_FAILURE)
                    break

                # If no problems, store results.
                integrator_data_ptr = &integration_solution_ptr.solution[0]
                # Need to make a copy because the solver pointers will be reallocated during the next solution.
                # Get storage pointer for this solution
                storage_by_y_ptr = storage_by_solution_ptr[solution_i]

                for y_i in range(num_ys):
                    for slice_i in range(layer_slices):
                        # Convert 2x real ys to 1x complex ys
                        storage_by_y_ptr[num_ys * slice_i + y_i] = cf_build_dblcmplx(
                            integrator_data_ptr[num_ys_dbl * slice_i + (2 * y_i)],
                            integrator_data_ptr[num_ys_dbl * slice_i + (2 * y_i) + 1]
                            )

                    # Store top most result for initial condition for the next layer
                    # slice_i should already be set to the top of this layer after the end of the previous loop.
                    uppermost_y_per_solution_ptr[solution_i * MAX_NUM_Y + y_i] = storage_by_y_ptr[num_ys * slice_i + y_i]
                
                # Decrement the shared pointer for the solution
                integration_solution.reset()
                    
            if error:
                # Error was encountered during integration
                solution_storage_ptr.success = False
                break

            # Prepare for next layer
            layer_below_num_sols     = num_sols
            layer_below_num_ys       = num_ys
            last_layer_upper_gravity = gravity_upper
            last_layer_upper_density = density_upper

        # Break out of success while loop
        break

    if error or not solution_storage_ptr.success:
        error = True
        solution_storage_ptr.success = False
        sprintf(message_ptr, 'RadialSolver.ShootingMethod:: Integration failed:\n\t%s.\n', integration_solution_ptr.message_ptr)
        solution_storage_ptr.set_message(message_ptr)
        # Decrement the shared pointer for the solution
        integration_solution.reset()
        if verbose or raise_on_fail:
            printf(message_ptr)
        if raise_on_fail:
            exit(EXIT_FAILURE)
    else:
        # No errors. Proceed with collapsing all sub-solutions into final full solution.
        strcpy(message_ptr, 'Integration completed for all layers. Beginning solution collapse.\n')

        for ytype_i in range(num_ytypes):
            sprintf(message_ptr, 'Collapsing radial solutions for "%d" solver.\n', ytype_i)

            # Reset variables for this solver
            bc_solution_info = -999
            constant_vector_ptr[0] = cmplx_NAN
            constant_vector_ptr[1] = cmplx_NAN
            constant_vector_ptr[2] = cmplx_NAN
            layer_above_lower_gravity   = NAN
            layer_above_lower_density   = NAN
            liquid_density_at_interface = NAN
            layer_above_type            = 9
            layer_above_is_static       = False
            layer_above_is_incomp       = False

            if verbose:
                printf(message_ptr)
            # Collapse the multiple solutions for each layer into one final combined solution.

            # Work from the surface to the core.
            for current_layer_i in range(num_layers):
                layer_i_reversed = num_layers - (current_layer_i + 1)

                # Pull out layer information.
                first_slice_index  = first_slice_index_by_layer_ptr[layer_i_reversed]
                layer_slices = num_slices_by_layer_ptr[layer_i_reversed]

                # Get solution and y information
                num_sols     = num_solutions_by_layer_ptr[layer_i_reversed]
                num_ys       = 2 * num_sols

                # Setup pointer array slices starting at this layer's beginning
                layer_radius_ptr    = &radius_array_ptr[first_slice_index]
                layer_density_ptr   = &density_array_ptr[first_slice_index]
                layer_gravity_ptr   = &gravity_array_ptr[first_slice_index]
                layer_bulk_mod_ptr  = &complex_bulk_array_ptr[first_slice_index]
                layer_shear_mod_ptr = &complex_shear_array_ptr[first_slice_index]

                # Get physical parameters at the top and bottom of the layer
                radius_lower  = layer_radius_ptr[0]
                density_lower = layer_density_ptr[0]
                gravity_lower = layer_gravity_ptr[0]
                bulk_lower    = layer_bulk_mod_ptr[0]
                shear_lower   = layer_shear_mod_ptr[0]
                radius_upper  = layer_radius_ptr[layer_slices - 1]
                density_upper = layer_density_ptr[layer_slices - 1]
                gravity_upper = layer_gravity_ptr[layer_slices - 1]

                # Get assumptions for layer
                layer_type      = layer_types_ptr[layer_i_reversed]
                layer_is_static = is_static_by_layer_ptr[layer_i_reversed]
                layer_is_incomp = is_incompressible_by_layer_ptr[layer_i_reversed]

                # Get full solutions for this layer
                storage_by_solution_ptr = main_storage_ptr[layer_i_reversed]

                # Get radial solution values at the top of the layer
                for solution_i in range(num_sols):
                    for y_i in range(num_ys):
                        uppermost_y_per_solution_ptr[solution_i * MAX_NUM_Y + y_i] = \
                            storage_by_solution_ptr[solution_i][(layer_slices - 1) * num_ys + y_i]

                if current_layer_i == 0:
                    # Working on surface (uppermost) layer -- Apply surface boundary conditions.
                    cf_apply_surface_bc(
                        constant_vector_ptr,  # Modified Variable
                        &bc_solution_info,  # Modified Variable
                        bc_pointer,
                        uppermost_y_per_solution_ptr,
                        surface_gravity,
                        G_to_use,
                        num_sols,
                        MAX_NUM_Y,
                        ytype_i,
                        layer_type,
                        layer_is_static,
                        layer_is_incomp
                        )

                    # Check that the boundary condition was successfully applied.
                    if bc_solution_info != 0:
                        error = True
                        sprintf(message_ptr, 'RadialSolver.ShootingMethod:: Error encountered while applying surface boundary condition. ZGESV code: %d.\nThe solutions may not be valid at the surface.\n', bc_solution_info)
                        solution_storage_ptr.set_message(message_ptr)
                        if verbose or raise_on_fail:
                            printf(message_ptr)
                        if raise_on_fail:
                            exit(EXIT_FAILURE)
                        break
                else:
                    # Working on interior layers. Will need to find the constants of integration based on the layer above.

                    cf_top_to_bottom_interface_bc(
                        constant_vector_ptr,  # Modified Variable
                        layer_above_constant_vector_ptr,
                        uppermost_y_per_solution_ptr,
                        gravity_upper, layer_above_lower_gravity,
                        density_upper, layer_above_lower_density,
                        layer_type, layer_above_type,
                        layer_is_static, layer_above_is_static,
                        layer_is_incomp, layer_above_is_incomp,
                        num_sols, MAX_NUM_Y
                        )

                # Use constant vectors to find the full y from all of the solutions in this layer
                cf_collapse_layer_solution(
                    solution_ptr,  # Modified Variable
                    constant_vector_ptr,
                    storage_by_solution_ptr,
                    layer_radius_ptr,
                    layer_density_ptr,
                    layer_gravity_ptr,
                    frequency,
                    first_slice_index,
                    layer_slices,
                    num_sols,
                    MAX_NUM_Y,
                    num_ys,
                    num_output_ys,
                    ytype_i,
                    layer_type,
                    layer_is_static,
                    layer_is_incomp
                    )

                # Setup for next layer
                layer_above_lower_gravity = gravity_lower
                layer_above_lower_density = density_lower
                layer_above_type          = layer_type
                layer_above_is_static     = layer_is_static
                layer_above_is_incomp     = layer_is_incomp

                if num_sols == 1:
                    layer_above_constant_vector_ptr[0] = constant_vector_ptr[0]
                    layer_above_constant_vector_ptr[1] = cmplx_NAN
                    layer_above_constant_vector_ptr[2] = cmplx_NAN
                elif num_sols == 2:
                    layer_above_constant_vector_ptr[0] = constant_vector_ptr[0]
                    layer_above_constant_vector_ptr[1] = constant_vector_ptr[1]
                    layer_above_constant_vector_ptr[2] = cmplx_NAN
                elif num_sols == 3:
                    layer_above_constant_vector_ptr[0] = constant_vector_ptr[0]
                    layer_above_constant_vector_ptr[1] = constant_vector_ptr[1]
                    layer_above_constant_vector_ptr[2] = constant_vector_ptr[2]
            
            # Ready for next y-type

    # Free memory
    # Deconstruct main solution pointer
    # Main storage pointers are structured like [current_layer_i][solution_i][y_i + slice_i]
    # Then main storage
    if not (main_storage_ptr is NULL):
        storage_by_solution_ptr = NULL
        storage_by_y_ptr = NULL
        for current_layer_i in range(num_layers):
            num_sols = num_solutions_by_layer_ptr[current_layer_i]
            if not (main_storage_ptr[current_layer_i] is NULL):
                for solution_i in range(num_sols):
                    if not (main_storage_ptr[current_layer_i][solution_i] is NULL):
                        free_mem(main_storage_ptr[current_layer_i][solution_i])
                        main_storage_ptr[current_layer_i][solution_i] = NULL
                
                free_mem(main_storage_ptr[current_layer_i])
                main_storage_ptr[current_layer_i] = NULL
        free_mem(main_storage_ptr)
        main_storage_ptr = NULL

    # Update solution status and return
    if error:
        solution_storage_ptr.success = False
        solution_storage_ptr.set_message(message_ptr)
    else:
        solution_storage_ptr.success = True
        solution_storage_ptr.set_message('RadialSolver.ShootingMethod:: completed without any noted issues.\n')


Content of stdout:
_cython_magic_d9edc1aa455da8db458c44f2d36472dc35d558be.cpp
C:\Users\joepr\anaconda3\envs\tpy6py311\Lib\site-packages\numpy\core\include\numpy\npy_1_7_deprecated_api.h(14) : Warning Msg: Using deprecated NumPy API, disable it with #define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
C:\Users\joepr\.ipython\cython\_cython_magic_d9edc1aa455da8db458c44f2d36472dc35d558be.cpp(30653): warning C4477: 'printf' : format string '%d' requires an argument of type 'int', but variadic argument 1 has type 'size_t'
C:\Users\joepr\.ipython\cython\_cython_magic_d9edc1aa455da8db458c44f2d36472dc35d558be.cpp(30653): note: consider using '%zd' in the format string
C:\Users\joepr\.ipython\cython\_cython_magic_d9edc1aa455da8db458c44f2d36472dc35d558be.cpp(30691): warning C4477: 'printf' : format string '%d' requires an argument of type 'int', but variadic argument 1 has type 'size_t'
C:\Users\joepr\.ipython\cython\_cython_magic_d9edc1aa455da8db458c44f2d36472dc35d558be.cpp(30691): note: consider 

In [5]:
%%cython -f -a
# distutils: language = c++
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True, initializedcheck=False

import numpy as np
cimport numpy as np
np.import_array()

cdef struct EOSOutput:
    double density
    double complex bulk_modulus
    double complex shear_modulus

from libcpp cimport bool as cpp_bool
from CyRK cimport PreEvalFunc

from libc.math cimport M_PI
cdef double PI_DBL = M_PI

cdef struct EOS_ODEInput:
    double G_to_use
    double planet_radius
    void* eos_input_ptr
    cpp_bool final_solve
    cpp_bool update_bulk
    cpp_bool update_shear

cdef void eos_diffeq(
        double* dy_ptr,
        double radius,
        double* y_ptr,
        const void* input_args,
        PreEvalFunc eos_function) noexcept nogil:
    """ Solve for EOS components as a function of radius. """

    # Cast input args to correct structure type
    cdef EOS_ODEInput* eos_input_ptr = <EOS_ODEInput*>input_args

    # Other constants
    cdef double grav_coeff = 4. * PI_DBL * eos_input_ptr.G_to_use

    # Update viscoelastic parameters using the user-provided equation of state
    cdef EOSOutput eos_output
    cdef EOSOutput* eos_output_ptr = &eos_output 
    eos_function(eos_output_ptr, radius, y_ptr, input_args)
    
    cdef double rho = eos_output.density
    
    # Solve for gravity and pressure
    if (radius <= 0.)  or (radius > eos_input_ptr.planet_radius):
        dy_ptr[0] = 0.0
        dy_ptr[1] = 0.0
    else:
        # Acceleration due to Gravity
        dy_ptr[0] = grav_coeff * rho - 2.0 * y_ptr[0] * (1.0 / radius)

        # Pressure
        dy_ptr[1] = -rho * y_ptr[0]

    # TODO: Track the static shear and bulk as well as the bulk and shear viscosity as additional outputs.
    if eos_input_ptr.final_solve:
        # There are two real dy/dt: gravity and pressure and then 9 additional parameters that are saved but
        # not used during integration but which the user may want for reference.
        dy_ptr[2] = eos_output.density

        dy_ptr[3] = eos_output.shear_modulus.real
        dy_ptr[4] = eos_output.shear_modulus.imag

        dy_ptr[5] = eos_output.bulk_modulus.real
        dy_ptr[6] = eos_output.bulk_modulus.imag


Content of stdout:
_cython_magic_75a0a7e3500411ab56fe80f30783ccbdc8a5b610.cpp
C:\Users\joepr\anaconda3\envs\tpy6py311\Lib\site-packages\numpy\core\include\numpy\npy_1_7_deprecated_api.h(14) : Warning Msg: Using deprecated NumPy API, disable it with #define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
C:\Users\joepr\anaconda3\envs\tpy6py311\Lib\site-packages\CyRK\cy\dense.cpp(119): warning C5051: attribute [[unlikely]] requires at least '/std:c++20'; ignored
C:\Users\joepr\anaconda3\envs\tpy6py311\Lib\site-packages\CyRK\cy\cysolution.cpp(53): warning C5051: attribute [[likely]] requires at least '/std:c++20'; ignored
C:\Users\joepr\anaconda3\envs\tpy6py311\Lib\site-packages\CyRK\cy\cysolution.cpp(68): warning C5051: attribute [[unlikely]] requires at least '/std:c++20'; ignored
C:\Users\joepr\anaconda3\envs\tpy6py311\Lib\site-packages\CyRK\cy\cysolution.cpp(98): warning C5051: attribute [[unlikely]] requires at least '/std:c++20'; ignored
C:\Users\joepr\anaconda3\envs\tpy6py311\Lib\site-pa